In [ ]:
import subprocess
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import concurrent.futures
import os
import certifi
import pickle
from Bio import Entrez, SeqIO, AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import CompoundLocation, ExactPosition, BeforePosition, AfterPosition
from io import StringIO
import glob
from tqdm.notebook import tqdm
import string
from ete3 import NCBITaxa
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from Bio.Data import CodonTable
import json
import copy
import threading
import sys
import tempfile
import numpy as np
import urllib
from bs4 import BeautifulSoup
import re
from collections import defaultdict
from Bio.Blast.Applications import NcbideltablastCommandline
from Bio.Blast import NCBIXML
from functools import partial

First we run Infernal's cmsearch of the group I intron covariance model (obtained from Rfam) against the entire NT database. We use the parameters given in Rfam for cmsearch, but changing the database size in Mbp to the size of NT (1.3 trillion bases).

In [ ]:
# Define the variables
num_cpus = 100
output_file_path = "/path/to/output/cmsearch_on_NT/results.tbl"
cm_model_path = "/path/to/cm/model.cm"
input_fasta_file_path = "/path/to/nt/database/nt"
path_to_cmsearch = "/path/to/cmsearch/bin/cmsearch"

# Define the command
command = f"{path_to_cmsearch} --cpu {num_cpus} --verbose --nohmmonly -E 1000 -Z 1300000 --tblout {output_file_path} {cm_model_path} {input_fasta_file_path}"

# Run the command
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()


We then read the output of cmsearch.

In [ ]:
def read_infernal_output(file_path):
    """Read an Infernal output file into a pandas DataFrame."""
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            fields = line.split(maxsplit=17)
            data.append(fields)
    df = pd.DataFrame(data)
    df.columns = ['target name', 'accession', 'query name', 'accession', 'mdl', 'mdl from', 'mdl to', 'seq from', 'seq to', 'strand', 'trunc', 'pass', 'gc', 'bias', 'score', 'E-value', 'inc', 'description of target']
    return df

infernal_NT_search_path = output_file_path  # Path to the Infernal output file

infernal_NT_search_all_hits = read_infernal_output(infernal_NT_search_path)

We then need to fetch the GenBank entries of all records that gave a cmsearch hit

In [ ]:
os.environ['SSL_CERT_FILE'] = certifi.where()

email = "your.email@email.com"  # Your email

def fetch_genbank_records_batch(email, batch_ids, max_tries=100):
    """Fetch GenBank records for a batch of IDs."""
    Entrez.email = email
    records = {}
    for _ in range(max_tries):
        try:
            handle = Entrez.efetch(db="nucleotide", id=batch_ids, rettype="gb", retmode="text")
            for record in SeqIO.parse(handle, "genbank"):
                records[record.id] = record
            handle.close()
            return records
        except:
            time.sleep(1)  # Wait for a second before retrying

def fetch_genbank_records(email, ids, batch_size=500, max_workers=10):
    """Fetch GenBank records for a list of IDs in batches using multiple threads."""
    records = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_genbank_records_batch, email, ids[i:i+batch_size]) for i in range(0, len(ids), batch_size)}
        for i, future in enumerate(concurrent.futures.as_completed(futures), 1):
            records.update(future.result())  # Update the records dictionary with the result of the future
            if i % 50 == 0:  # Every 50 batches
                with open('genbank_records.pkl', 'wb') as f:
                    pickle.dump(records, f)  # Save the records to a file, to be able to restore partial results
                print(f"Saved results after processing {i} batches")
    return records

ids = list(set(infernal_NT_search_all_hits["target name"].tolist()))  # Remove duplicates before passing to function

infernal_NT_search_genbank_recs = fetch_genbank_records(email, ids, batch_size=50, max_workers=10)

# save the final results 
with open('genbank_records.pkl', 'wb') as f:
    pickle.dump(infernal_NT_search_genbank_recs, f) # note this will require large amount of memory to write and also to read later

In [ ]:
with open('genbank_records.pkl', 'rb') as f:
    infernal_NT_search_genbank_recs = pickle.load(f)

Some keys will have records with undefined sequences because their sequences are too large and must be explicitly requested as fasta. So we find these and handle them separately. We save these separately as they are very large

In [ ]:
genbank_recs_with_undefined_seqs = [key for key, value in infernal_NT_search_genbank_recs.items() if value.seq.defined is False]

batch_size = 100
genbank_ids_to_refetch = genbank_recs_with_undefined_seqs
seqs_of_records_with_undefined_seqs = {}

for i in range(0, len(genbank_ids_to_refetch), batch_size):
    print(f"Processing batch {i+1}-{i+batch_size}")
    batch_ids = genbank_ids_to_refetch[i:i+batch_size]
    handle = Entrez.efetch(db="nucleotide", id=batch_ids, rettype="fasta", retmode="text")
    records = list(SeqIO.parse(handle, "fasta"))  # Convert iterator to list
    with open(f'records_with_previously_undefined_seqs_batch_{i+1}_{i+batch_size}.fasta', 'w') as f:
        SeqIO.write(records, f, "fasta")  # Write records to file
    for record in records:
        seqs_of_records_with_undefined_seqs[record.id] = record.seq
    print(f"Finished processing batch {i+1}-{i+batch_size}")

If required we can restore the full sequences from the fasta files.

In [ ]:
seqs_of_records_with_undefined_seqs = {}
merged_records = []

# Find all files that match the pattern
for file_name in glob.glob("records_with_previously_undefined_seqs_batch_*.fasta"):
    # Open the file and parse the records
    with open(file_name, "r") as handle:
        records = list(SeqIO.parse(handle, "fasta"))
    # Add the records to the merged_records list
    merged_records.extend(records)

# Add the sequences to the seqs_of_records_with_undefined_seqs dictionary
for record in merged_records:
    seqs_of_records_with_undefined_seqs[record.id] = record.seq

len(seqs_of_records_with_undefined_seqs)

We now want to filter the intron hits to keep only those for which we can find reliable boundaries.
In order to do so, we look for features labeled as intron in the corresponding GenBank entries that overlap by a minimum overlap threshold with the cmsearch hit.
If there are no such features, we look for features with a location of type CompoundLocation (those shown as join(...)) and look for a gap between exons that overlaps with the cmsearch hit.
We perform the search in both the direct and the complementary strand.

In [ ]:
overlap_threshold = 50
intron_boundaries_genbank_infernal_NT_search = {}
rows_with_undefined_seqs = []

for index, row in tqdm(infernal_NT_search_all_hits.iterrows(), total=infernal_NT_search_all_hits.shape[0]):
    infernal_intron_start = int(row["seq from"])
    infernal_intron_end = int(row["seq to"])
    infernal_boundaries = (infernal_intron_start, infernal_intron_end)
    infernal_hit_strand = row["strand"]
    genbank_ID = row["target name"]
    if genbank_ID == 'MT229979.1':
        # skip this iteration since this seems to be a strange record that was removed from GenBank
        continue
    genbank_record = infernal_NT_search_genbank_recs[genbank_ID]
    if genbank_record.seq.defined:
        genbank_sequence_raw = genbank_record.seq
    else:
        genbank_sequence_raw = seqs_of_records_with_undefined_seqs[genbank_ID]
    genbank_sequence = genbank_sequence_raw.replace('U', 'T')
    genbank_features = genbank_record.features 
    new_intron_boundaries = None
    source = None
    evalue = row["E-value"]
    bitScore = row["score"]
    candidate_reliable_introns = []
    if(infernal_intron_start < infernal_intron_end):
        infernal_hit_sequence_region = genbank_sequence[(infernal_intron_start - 1):(infernal_intron_end)]
        infernal_intron_range = range(infernal_intron_start, infernal_intron_end)
        for feature in genbank_features:
            if feature.type == 'intron':
            # Check for 'intron' features
                if isinstance(feature.location, CompoundLocation):
                    for i in range(len(feature.location.parts) - 1):
                        start_first_intron = feature.location.parts[i].start
                        end_first_intron = feature.location.parts[i].end
                        candidate_intron_range = range(int(start_first_intron), int(end_first_intron))
                        overlap = len(set(infernal_intron_range) & set(candidate_intron_range))
                        if overlap >= overlap_threshold and candidate_intron_range[0] >= 1 and candidate_intron_range[1] < (len(genbank_sequence) - 1):
                            source = 'intronCompoundLocation'
                            new_intron_start = start_first_intron - 1
                            new_intron_end = end_first_intron
                            candidate_intron_sequence = genbank_sequence[new_intron_start:(new_intron_end + 1)]
                            candidate_reliable_introns.append((genbank_ID, new_intron_start, new_intron_end, candidate_intron_sequence, overlap, source, infernal_intron_start, infernal_intron_end, evalue, bitScore, infernal_hit_sequence_region))
                elif isinstance(feature.location.start, ExactPosition) and isinstance(feature.location.end, ExactPosition):
                    candidate_intron_range = range(int(feature.location.start), int(feature.location.end))
                    overlap = len(set(infernal_intron_range) & set(candidate_intron_range))
                    if overlap >= overlap_threshold and candidate_intron_range[0] >= 1 and candidate_intron_range[1] < (len(genbank_sequence) - 1):
                        source = 'intron'
                        new_intron_start = int(feature.location.start) - 1
                        new_intron_end = int(feature.location.end)
                        candidate_intron_sequence = genbank_sequence[new_intron_start:(new_intron_end + 1)]
                        candidate_reliable_introns.append((genbank_ID, new_intron_start, new_intron_end, candidate_intron_sequence, overlap, source, infernal_intron_start, infernal_intron_end, evalue, bitScore, infernal_hit_sequence_region))
            elif isinstance(feature.location, CompoundLocation):
            # Check for 'compound location' features
                for i in range(len(feature.location.parts) - 1):
                    # Get the end of the first part and the start of the second part
                    end_first_part = feature.location.parts[i].end
                    start_second_part = feature.location.parts[i+1].start
                    candidate_intron_range = range(int(end_first_part), int(start_second_part))
                    overlap = len(set(infernal_intron_range) & set(candidate_intron_range))
                    if overlap >= overlap_threshold and start_second_part - end_first_part < 3000:
                        source = 'compoundLocation'
                        new_intron_start = end_first_part - 1
                        new_intron_end = start_second_part
                        candidate_intron_sequence = genbank_sequence[new_intron_start:(new_intron_end + 1)]
                        candidate_reliable_introns.append((genbank_ID, new_intron_start, new_intron_end, candidate_intron_sequence, overlap, source, infernal_intron_start, infernal_intron_end, evalue, bitScore, infernal_hit_sequence_region))
    else:
        infernal_hit_sequence_region = genbank_sequence[(infernal_intron_end - 1):(infernal_intron_start)].reverse_complement()
        infernal_intron_range = range(infernal_intron_start, infernal_intron_end, -1)
        for feature in genbank_features:
            if feature.type == 'intron' and feature.location.strand == -1:
            # Check for 'intron' features
                if isinstance(feature.location.start, ExactPosition) and isinstance(feature.location.end, ExactPosition):
                    candidate_intron_range = range(int(feature.location.end), int(feature.location.start), -1)
                    overlap = len(set(infernal_intron_range) & set(candidate_intron_range))
                    # It seems that ALWAYS int(feature.location.end) > int(feature.location.start) in these cases of features of type intron in strand -1
                    if overlap >= overlap_threshold and candidate_intron_range[1] >= 1 and candidate_intron_range[0] < (len(genbank_sequence) - 1): #note indexes for accession on the range are reversed
                        source = 'intron_reverseStrand'
                        new_intron_start = int(feature.location.start) -1
                        new_intron_end = int(feature.location.end)
                        candidate_intron_sequence = genbank_sequence[new_intron_start:(new_intron_end + 1)]
                        # Reverse complement the candidate_intron_sequence
                        reverse_complement_sequence = Seq(candidate_intron_sequence).reverse_complement()
                        candidate_reliable_introns.append((genbank_ID, new_intron_start, new_intron_end, reverse_complement_sequence, overlap, source, infernal_intron_start, infernal_intron_end, evalue, bitScore, infernal_hit_sequence_region))
            elif isinstance(feature.location, CompoundLocation) and feature.location.strand == -1:
                for i in range(len(feature.location.parts) - 1):
                    start_first_part = feature.location.parts[i].start
                    end_second_part = feature.location.parts[i+1].end
                    candidate_intron_range = range(int(start_first_part), int(end_second_part), -1)
                    overlap = len(set(infernal_intron_range) & set(candidate_intron_range))
                    if overlap >= overlap_threshold and start_first_part - end_second_part < 3000:
                        source = 'compoundLocation_reverseStrand'
                        new_intron_start = end_second_part
                        new_intron_end = start_first_part
                        candidate_intron_sequence = genbank_sequence[new_intron_start:(new_intron_end + 1)]
                        reverse_complement_sequence = Seq(candidate_intron_sequence).reverse_complement()
                        candidate_reliable_introns.append((genbank_ID, new_intron_start, new_intron_end, reverse_complement_sequence, overlap, source, infernal_intron_start, infernal_intron_end, evalue, bitScore, infernal_hit_sequence_region))
    if len(candidate_reliable_introns) >= 1:
        if(len(candidate_reliable_introns) == 1):
            selected_intron = candidate_reliable_introns[0]
        else:
            # Sort the candidate introns by overlap in descending order
            candidate_reliable_introns.sort(key=lambda x: x[4], reverse=True)
            # Get the highest overlap value
            highest_overlap = candidate_reliable_introns[0][4]
            # Filter the candidate introns with the highest overlap value
            highest_overlap_introns = [intron for intron in candidate_reliable_introns if intron[4] == highest_overlap]
            # Check if all the sequences are the same
            sequences = [intron[3] for intron in highest_overlap_introns]
            if len(set(sequences)) == 1:
                selected_intron = highest_overlap_introns[0]
            else:
                # Check which introns have 'T' as their first letter
                introns_with_T = [intron for intron in highest_overlap_introns if intron[3][0] == 'T']
                if len(introns_with_T) > 0:
                    selected_intron = introns_with_T[0]
                else:
                    selected_intron = highest_overlap_introns[0]
    else:
        selected_intron = None
    if genbank_ID in intron_boundaries_genbank_infernal_NT_search:
        if selected_intron is not None:
            # then we have to append the selected intron to the list of introns
            # check if the value is a list or a single value
            if isinstance(intron_boundaries_genbank_infernal_NT_search[genbank_ID], list):
                intron_boundaries_genbank_infernal_NT_search[genbank_ID].append(selected_intron)
            else:
                intron_boundaries_genbank_infernal_NT_search[genbank_ID] = [intron_boundaries_genbank_infernal_NT_search[genbank_ID], selected_intron]
    else:
        intron_boundaries_genbank_infernal_NT_search[genbank_ID] = selected_intron

intron_boundaries_genbank_infernal_NT_search_found = {key: value for key, value in intron_boundaries_genbank_infernal_NT_search.items() if value is not None}

We then select only introns that start with T (keeping in mind that the first nucleotide of each extracted intron is the last nucleotide of the exon), and we add the sequence length to each intron

In [ ]:
intron_boundaries_genbank_infernal_NT_search_found_startsWithT = {}
for key, value in intron_boundaries_genbank_infernal_NT_search_found.items():
    if isinstance(value, list):
        # Filter out introns that don't start with a "T"
        good_introns = [intron for intron in value if intron is not None and intron[3].startswith('T')]
        if good_introns:
            # If there's only one good intron, store it as a tuple instead of a list
            if len(good_introns) == 1:
                intron_boundaries_genbank_infernal_NT_search_found_startsWithT[key] = good_introns[0]
            else:
                intron_boundaries_genbank_infernal_NT_search_found_startsWithT[key] = good_introns
    else:
        # If the intron doesn't start with a "T", skip it
        if not value[3].startswith('T'):
            continue
        intron_boundaries_genbank_infernal_NT_search_found_startsWithT[key] = value

intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths = {}
for key, value in intron_boundaries_genbank_infernal_NT_search_found_startsWithT.items():
    if isinstance(value, list):
        new_value = []
        for intron in value:
            intron_length = len(intron[3])
            new_intron = intron + (intron_length,)
            new_value.append(new_intron)
        intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths[key] = new_value
    else:
        intron_length = len(value[3])
        new_value = value + (intron_length,)
        intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths[key] = new_value

Additionally, there seems to be a single abnormal occurrence of an intron too long due to it having a compound feature annotated with a gap of nearly 100000 nucleotides. We simply remove this instance with a very high max length threshold

In [ ]:
# Specify the maximum length
max_length = 10000

intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs = {}
for key, value in intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths.items():
    if isinstance(value, list):
        # Filter out introns that are too long
        good_introns = [intron for intron in value if intron[-1] <= max_length]
        if good_introns:
            # If there's only one good intron, store it as a tuple instead of a list
            if len(good_introns) == 1:
                intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs[key] = good_introns[0]
            else:
                intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs[key] = good_introns
    else:
        # If the intron is too long, skip it
        if value[-1] > max_length:
            continue
        intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs[key] = value

At this point, the structure that we have is a dictionary where each key is a GenBank entry that contains at least 1 group I intron with reliable boundaries. If there is a single such intron, the value is a tuple describing the intron. If there are multiple, the value is a list of such tuples. We now want to extend each intron tuple with additional metadata: organism name, nucleic acid molecular type, taxonomy ID and subcellular location (organelle). We can extract these from the source feature of each GenBank entry. 

In [ ]:
intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend = {}
for key, value in intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs.items():
    # Extract the additional data from the 'source' feature
    source_feature = next(feature for feature in infernal_NT_search_genbank_recs[key].features if feature.type == 'source')
    organism = source_feature.qualifiers.get('organism', [''])[0]
    mol_type = source_feature.qualifiers.get('mol_type', [''])[0]
    db_xref = source_feature.qualifiers.get('db_xref', [''])
    # db_xref might be a list, so we want to take the element that contains the substring taxon
    if isinstance(db_xref, list):
        db_xref = next(xref for xref in db_xref if 'taxon' in xref)
    organelle = source_feature.qualifiers.get('organelle', [''])[0]
    if isinstance(value, list):
        new_value = []
        for intron in value:
            new_intron = intron + (organism, mol_type, db_xref, organelle)
            new_value.append(new_intron)
        intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend[key] = new_value
    else:
        new_value = value + (organism, mol_type, db_xref, organelle)
        intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend[key] = new_value

We now flatten the dictionary so that each entry has a tuple as value. When required, we append _i to the keys with more than 1 intron.

In [ ]:
intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened = {}
for key, value in intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend.items():
    if isinstance(value, list):
        for i, intron in enumerate(value, start=1):
            new_key = f"{key}_{i}"
            intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened[new_key] = intron
    else:
        intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened[key] = value

We can save the dictionary for future use.

In [ ]:
with open('intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened.pkl', 'wb') as f:
    pickle.dump(intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened, f)

In [ ]:
with open('intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened.pkl', 'rb') as f:
    intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened = pickle.load(f)

Each value in the dictionary of introns is a tuple describing the intron. We convert these to dictionaries to make clearer what each element is.

In [ ]:
intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict = {}

for key, value in intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened.items():
    genbank_id, start_position, end_position, sequence, overlap, boundariesSource, infernal_hit_start_position, infernal_hit_end_position, e_value, bit_score, infernal_hit_sequence_region, length, organism, molecule_type, tax_id, organelle = value
    if organelle == "mitochondrion":
        organelle = "mitochondria"
    entry_dict = {
        'GenBankID': genbank_id,
        'startPosition': start_position,
        'endPosition': end_position,
        'sequence': sequence,
        'overlapWithInfernalHit': overlap,
        'boundariesSource': boundariesSource,
        'length': length,
        'organism': organism,
        'moleculeType': molecule_type,
        'taxID': tax_id,
        'organelle': organelle,
        'infernalHitStartPosition': infernal_hit_start_position,
        'infernalHitEndPosition': infernal_hit_end_position,
        'infernalHitEValue': float(e_value),
        'infernalHitBitScore': float(bit_score),
        'infernalHitRegionSequence': infernal_hit_sequence_region
        
    }
    intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict[key] = entry_dict

Through manual inspection of entries with strange taxonomy, we found 3 outliers that require manual fixing.

In [ ]:
# Firstly, there is a case where authors seem to have by mistake assigned taxid of Olea gastropod genus,
# whereas it should be the olive Olea as indicated by the organism name. So we are going to fix it manually
# the corrected data are taken from Table S1 of the corresponding paper
MT560017_1_entry = intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict['MT560017.1']
MT560017_1_entry["organism"] = "Olea sp. POC544315"
MT560017_1_entry["taxID"] = "taxon:2813885"

In [ ]:
# then we need to fix 2 entries that are annotated as artificial DNA because they were deposited as clones into vectors
# First entry for K03428.1
K03428_1_entry = intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict['K03428.1']
K03428_1_entry["organism"] = "Tetrahymena thermophila"
K03428_1_entry["taxID"] = "taxon:5911"

# and then entry JN563930.1
JN563930_1_entry = intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict['JN563930.1']
JN563930_1_entry["organism"] = "Nicotiana undulata"
JN563930_1_entry["taxID"] = "taxon:118713"
JN563930_1_entry["organelle"] = "plastid:chloroplast"

In [ ]:
# Extract the triplets
triplets = [(key, value['GenBankID'], value['startPosition'], value['endPosition']) for key, value in intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict.items()]

# Group the triplets
groups = defaultdict(list)
for key, genbank_id, start_position, end_position in triplets:
    groups[(genbank_id, start_position, end_position)].append(key)

group_lengths = {key: len(value) for key, value in groups.items()}

groups_with_one_element = {key: value for key, value in groups.items() if len(value) == 1}
groups_with_more_than_one_element = {key: value for key, value in groups.items() if len(value) > 1}

intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict_noDuplicates = {}
keys_groups_with_one_element = [key for sublist in groups_with_one_element.values() for key in sublist]

for key in keys_groups_with_one_element:
    intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict_noDuplicates[key] = intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict[key]

for triple, keys in groups_with_more_than_one_element.items():
    subdictionary = {key: intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict[key] for key in keys}
    infernal_start_positions = [value['infernalHitStartPosition'] for value in subdictionary.values()]
    infernal_end_positions = [value['infernalHitEndPosition'] for value in subdictionary.values()]
    infernal_boundaries = list(zip(infernal_start_positions, infernal_end_positions))
    infernal_boundaries_unique = list(set(infernal_boundaries))
    infernal_start_positions_unique, infernal_end_positions_unique = zip(*infernal_boundaries_unique)
    entry_to_add = subdictionary[keys[0]]
    entry_to_add["infernalHitStartPosition"] = infernal_start_positions_unique
    entry_to_add["infernalHitEndPosition"] = infernal_end_positions_unique
    intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict_noDuplicates[keys[0]] = entry_to_add

This gives us the 42459 group I introns with reliable boundaries. We now want to classify the introns into organism types based on the taxonomy ID. We use a local copy of NCBI's taxonomy database for it

In [ ]:
ncbi = NCBITaxa()
# Update the local database
ncbi.update_taxonomy_database()

In [ ]:
def classify_intron(key_value):
    ncbi = NCBITaxa()
    key, value = key_value
    taxid_number = value.split(":")[1]
    recovery_successful = False
    updated_taxid = False
    try:
        lineage = ncbi.get_lineage(taxid_number)
        recovery_successful = True
    except Exception as e:
        # then we are going to try to do the organism search from organisn name
        # first we get the organism name from the flattened dictionary using the same key
        organism_name = intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict_noDuplicates[key]['organism']
        # then if the organism name contains the substring "aff. ", we remove it
        if "aff. " in organism_name:
            organism_name = organism_name.replace("aff. ", "")
        # then we are going to search for the organism name in the NCBI taxonomy database
        taxid_number = ncbi.get_name_translator([organism_name])
        taxid_number = taxid_number[organism_name][0]
        updated_taxid = True
        # and then we try with this new taxid_number, using error handling in case it is again not found
        try:
            lineage = ncbi.get_lineage(taxid_number)
            recovery_successful = True
        except Exception as e:
            print(f"No result found for organism name: {organism_name}. Error: {e}")
            category = "other"

    if recovery_successful:
        names = ncbi.get_taxid_translator(lineage)
        names_values = names.values()
        lineages_dict[key] = names_values
    else:
        # this should not happen for any key
        lineages_dict[key] = None
        print(f"No result found for organism name: {organism_name}. Error: {e}")
        category = "other"
    if "Bacteria" in names_values:
        category = "bacteria"
    elif 'Viruses' in names_values:
        category = "virus"
    elif "Eukaryota" in names_values:
        if "Viridiplantae" in names_values or 'Diphylleia' in names_values:
            category = "plants"
        elif "Mollusca" in names_values:
            category = "molluscs"
        elif "Fungi" in names_values:
            category = "fungi"
        elif "Ciliophora" in names_values:
            category = "ciliates"
        elif "Oomycota" in names_values:
            category = "oomycetes"
        elif 'Acanthamoeba' in names_values or 'Amoebidium' in names_values or 'Dictyostelia' in names_values or 'Dictyostelium' in names_values or 'Heterostelium pallidum' in names_values or 'Myxogastria' in names_values or 'Physariida' in names_values or any("amoeba" in element for element in names_values) or 'Amoebozoa' in names_values or 'Nuclearia' in names_values:
            category = "amoebae"
        elif 'Bacillariophyta' in names_values:
            category = "diatoms"
        elif 'Chlorarachniophyceae' in names_values:
            category = "green algae"
        elif 'Choanoflagellata' in names_values:
            category = "choanoflagellates"
        elif 'Cryptophyceae' in names_values:
            category = "cryptophytes"
        elif 'Cyanophora' in names_values:
            category = "glaucophytes"
        elif 'Euglenida' in names_values:
            category = "euglenids"
        elif 'Eustigmatophyceae' in names_values:
            category = "eustigmatophytes"
        elif 'Heterolobosea' in names_values:
            category = "percolozoa"
        elif 'Heteromitidae' in names_values:
            category = "cercomonads"
        elif 'Phaeophyceae' in names_values or 'Schizocladia' in names_values:
            category = "brown algae"
        elif 'Plasmodiophorida' in names_values:
            category = "plasmodiophores"
        elif 'Porifera' in names_values:
            category = "sponges"
        elif 'Rhodophyta' in names_values:
            category = "red algae"
        elif 'Xanthophyceae' in names_values:
            category = "yellow-green algae"
        elif 'Hexacorallia' in names_values:
            category = "corals"
        elif 'Insecta' in names_values:
            category = "insects"
        elif 'Vertebrata' in names_values:
            category = "vertebrates"
        elif 'Cercozoa' in names_values:
            category = "cercozoans"
        elif 'Centroplasthelida' in names_values:
            category = "centrohelids"
        elif 'Placozoa' in names_values:
            category = "placozoans"
        else:
            category = "other"
        # note that many of 
    else:
        category = "other"
    entry_to_update = intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict_noDuplicates[key]
    extended_entry = entry_to_update.copy()
    extended_entry["organismType"] = category
    extended_entry["lineage"] = list(names_values)
    if updated_taxid:
        extended_entry["taxID"] = taxid_number
    return key, extended_entry

classified_introns = {}
lineages_dict = {}
# Extract all taxon IDs
taxon_ids = {key: entry['taxID'] for key, entry in intron_boundaries_genbank_infernal_NT_search_found_startsWithT_plus_lengths_removeTooLongs_orgExtend_flattened_dict_noDuplicates.items() if entry['taxID']}

with ThreadPoolExecutor() as executor:
    future_to_key = {executor.submit(classify_intron, item): item[0] for item in taxon_ids.items()}
    for future in tqdm(concurrent.futures.as_completed(future_to_key)):
        key = future_to_key[future]
        try:
            result = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (key, exc))
        else:
            classified_introns[result[0]] = result[1]

We can now save this dictionary with categories of organisms as well. It also contains the entire taxonomic lineages of each organism.

In [ ]:
with open('classified_introns.pkl', 'wb') as f:
    pickle.dump(classified_introns, f)

In [ ]:
with open('classified_introns.pkl', 'rb') as f:
    classified_introns = pickle.load(f)

We now make a barplot with the number of introns in each organism group. For this plot, we group all protists other than amoebae under category "protists". We group corals and sponges together also

In [ ]:
# Extract the organism types
organism_types = [entry['organismType'] for entry in classified_introns.values()]

# Count the occurrences of each organism type
counts = Counter(organism_types)

# Create a new dictionary where we group the categories for other protists
grouped_counts = {'plants': 0, 'fungi': 0, 'amoebae': 0, 'virus': 0, 'bacteria': 0, 'corals/sponges': 0, 'other protists': 0}
for organism_type, count in counts.items():
    if organism_type in grouped_counts:
        grouped_counts[organism_type] += count
    elif organism_type in ['corals', 'sponges']:
        grouped_counts['corals/sponges'] += count
    else:
        grouped_counts['other protists'] += count

# Sort the dictionary by the frequencies, except 'other protists' which should be last
sorted_counts = {k: v for k, v in sorted(grouped_counts.items(), key=lambda item: (-item[1], item[0] == 'other protists'))}

# Create the barplot

plt.rc('font', family='Arial', size=9)
plt.figure(figsize=(5.2/2.54, 5.2/2.54))

plt.bar(sorted_counts.keys(), sorted_counts.values())
plt.xlabel('Organism Type')
plt.ylabel('Frequency')

# Rotate x-axis labels
plt.xticks(rotation=90)
plt.savefig('Figure1b.png', dpi=300, bbox_inches='tight')

plt.show()

Now let's make a similar barplot but excluding plants and fungi

In [ ]:
# Create a new dictionary excluding 'plants' and 'fungi'
filtered_counts = {k: v for k, v in counts.items() if k not in ['plants', 'fungi']}

# Create the barplot
plt.rc('font', family='Arial', size=8)
plt.figure(figsize=(10/2.54, 5.2/2.54))

plt.bar(filtered_counts.keys(), filtered_counts.values())
plt.xlabel('Organism Type')
plt.ylabel('Frequency')

# Rotate x-axis labels
plt.xticks(rotation=90)
plt.savefig('Figure1c.png', dpi=300, bbox_inches='tight')
plt.show()

Now we are going to identify putative homing endonucleases in the introns. We first extract all possible ORFs of a minimum length in all reading frames, taking into consideration that translation to protein should be done with the appropriate genetic code taking into account organism and subcellular location. The correct genetic code also defines with which codons a candidate ORF can start and end. First, we will get the correct translation tables from NCBI taxonomies and parse them.

In [ ]:
def get_ncbi_taxonomy_translation_tables(taxid):
    requestUrl = "https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id=" + taxid
    response = urllib.request.urlopen(requestUrl)
    parsedResponse = BeautifulSoup(response, 'html.parser')
    a_tags = parsedResponse.find_all('a')
    genetic_codes = [(tag.previous_sibling + tag.text) for tag in a_tags if 'Translation table' in tag.text]
    return genetic_codes

translation_tables_from_NCBI_taxonomy = {}

for key, value in tqdm(classified_introns.items()):
    taxid = value["taxID"].split(":")[1]
    genetic_codes = get_ncbi_taxonomy_translation_tables(taxid)
    translation_tables_from_NCBI_taxonomy[key] = genetic_codes

In [ ]:
with open('translation_tables_from_NCBI_taxonomy.pkl', 'wb') as f:
    pickle.dump(translation_tables_from_NCBI_taxonomy, f)

In [ ]:
with open('translation_tables_from_NCBI_taxonomy.pkl', 'rb') as f:
    translation_tables_from_NCBI_taxonomy = pickle.load(f)

In [ ]:
translation_tables_from_NCBI_taxonomy_parsed = {}

for key, value in translation_tables_from_NCBI_taxonomy.items():
    # initialize a dictionary with keys general, mitochondrial, plastidic
    translation_tables_from_NCBI_taxonomy_parsed[key] = {"general": None, "mitochondrial": None, "plastidic": None}
    for item in value:
        code_category, translation_table = item.split(":")
        genetic_code_number = re.search(r'Translation table (\d+)', translation_table).group(1)
        if code_category == "Genetic code":
            translation_tables_from_NCBI_taxonomy_parsed[key]["general"] = int(genetic_code_number)
        elif code_category == "Mitochondrial genetic code":
            translation_tables_from_NCBI_taxonomy_parsed[key]["mitochondrial"] = int(genetic_code_number)
        elif code_category == "Plastid genetic code":
            translation_tables_from_NCBI_taxonomy_parsed[key]["plastidic"] = int(genetic_code_number)

In [ ]:
from Bio.Seq import Seq
from Bio.Data import CodonTable

def find_orfs(sequence, frame, genetic_code, min_length):
    start_codons = CodonTable.unambiguous_dna_by_id[genetic_code].start_codons
    if frame < 3:
        sequence_frame = sequence[frame:]
    else:
        sequence_frame = str(Seq(sequence).reverse_complement())[frame-3:]
    protein = str(Seq(sequence_frame).translate(table=genetic_code, to_stop=False))
    orfs = []
    for i in range(0, len(sequence_frame), 3):
        if sequence_frame[i:i+3] in start_codons:
            protein_start = i // 3
            for j in range(protein_start, len(protein)):
                if protein[j] == "*":
                    orf = protein[protein_start:j]
                    if len(orf) >= min_length:
                        orfs.append({
                            'sequence': orf,
                            'startInIntronSeq': frame + i,
                            'endInIntronSeq': frame + i + len(orf) * 3,
                            'geneticCode': genetic_code,
                            'readingFrame': frame
                        })
                    break
    # Remove ORFs that are fully contained within another ORF
    orfs = [orf1 for orf1 in orfs if not any(orf2['startInIntronSeq'] <= orf1['startInIntronSeq'] and orf2['endInIntronSeq'] >= orf1['endInIntronSeq'] for orf2 in orfs if orf2 != orf1)]
    return orfs

In [ ]:
min_length = 120
orf_dict_all_genetic_codes = {}
all_possible_genetic_codes = [1,2,3,4,5,6,9,10,11,12,13,14,15,16,21,22,23,24,25,26,27,28,29,30,31,33]

def get_intron_orfs(item):
    key, value = item
    orf_dict = {}
    for genetic_code in all_possible_genetic_codes:
        orf_dict[genetic_code] = {}
        for frame in range(6):
            orf_dict[genetic_code][frame] = find_orfs(value['sequence'], frame, genetic_code, min_length)
    return key, orf_dict

with concurrent.futures.ProcessPoolExecutor() as executor:
    orf_dict_all_genetic_codes = dict(executor.map(get_intron_orfs, classified_introns.items()))

In [ ]:
with open('orf_dict_all_genetic_codes.pkl', 'rb') as f:
    orf_dict_all_genetic_codes = pickle.load(f)

In [ ]:
with open('orf_dict_all_genetic_codes.pkl', 'wb') as f:
    pickle.dump(orf_dict_all_genetic_codes, f)

In [ ]:
with open("orfs_allIntrons_all_genetic_codes.fasta", "w") as output_handle:
    for key, value in orf_dict_all_genetic_codes.items():
        for genetic_code, frames in value.items():
            for frame, orfs in frames.items():
                for i, orf in enumerate(orfs):
                    sequence = Seq(orf['sequence'])
                    record = SeqRecord(sequence, id=f"{key}_allPossibleTranslations_geneticCode_{genetic_code}_frame_{frame}_orf_{i}", description="")
                    SeqIO.write(record, output_handle, "fasta")

We now make a run_interproscan.sh file that will be executed to run InterProScan on ORFs found on all introns with any genetic code. The file should be something like this:
    
```bash
    #!/bin/bash
    /path/to/interproscan.sh -i orfs_allIntrons_all_genetic_codes.fasta -f json -o orfs_allIntrons_all_genetic_codes.json -cpu 128 -dp
```

In [ ]:
# Run the shell script
command = ["bash", "run_interproscan.sh"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

if process.returncode != 0:
    print(f"InterProScan failed with error message:\n{stderr.decode()}")
else:
    print("InterProScan for all introns with all genetic codes completed successfully.")

In [ ]:
# Parse the InterProScan results and add them to a dictionary
interpro_results_allGeneticCodes = {}
with open('orfs_allIntrons_all_genetic_codes.json') as json_file:
    data = json.load(json_file)
    for protein in data['results']:
        if len(protein["matches"]) == 0:
            # then move on to next
            continue
        # first lets get the key (intronID), which will be the capture group containing all characters from the beginning until the substring _allPossible
        # but we need to iterate over all elements in protein['xref'], since sequences are indexed by sequence
        for i in range(len(protein['xref'])):
            key = re.match(r"(.*)_allPossibleTranslations", protein['xref'][i]['id']).group(1)
            id_parts = protein['xref'][0]['id'].split('_')
            orf_index = int(id_parts[-1])
            frame = int(id_parts[-3])
            geneticCode = int(id_parts[-5])
            if key not in interpro_results_allGeneticCodes:
                interpro_results_allGeneticCodes[key] = {}
            if geneticCode not in interpro_results_allGeneticCodes[key]:
                interpro_results_allGeneticCodes[key][geneticCode] = {}
            if frame not in interpro_results_allGeneticCodes[key][geneticCode]:
                interpro_results_allGeneticCodes[key][geneticCode][frame] = {}
            interpro_results_allGeneticCodes[key][geneticCode][frame][orf_index] = protein['matches']

Now we want to make a new dictionary where we only keep introns for which an ORF likely to be a homing endonuclease was found. We also add the InterProScan hits for these cases. It is possible that some introns might contain multiple homing endonucleases.

In [ ]:
def check_substrings(d):
    # Check if the current dictionary contains the substrings
    if any("endonuc" in value.lower() or "homing" in value.lower() or "nuclease" in value.lower() 
           for value in d.values() if isinstance(value, str)):
        return True
    # Recursively check the nested dictionaries
    return any(check_substrings(value) for value in d.values() if isinstance(value, dict))

# Make a deep copy of the orf_dict_all_genetic_codes dictionary
homingEndonucleases_dict = copy.deepcopy(orf_dict_all_genetic_codes)
discarded_ORFs_dict = copy.deepcopy(orf_dict_all_genetic_codes)

# Iterate over each intron in the copied dictionary
for intron, genetic_codes in list(homingEndonucleases_dict.items()):  # Use list to allow modifying the dictionary during iteration
    # For each intron, iterate over each frame
    for genetic_code, frames in list(genetic_codes.items()):  # Use list to allow modifying the dictionary during iteration
        for frame, orfs in list(frames.items()):  # Use list to allow modifying the dictionary during iteration
            # Create a new list that only includes the ORFs you want to keep
            new_orfs = []
            # For each frame, iterate over each ORF
            for i, orf in enumerate(orfs):
                # Check if the ORF has any InterProScan hits
                if intron in interpro_results_allGeneticCodes and genetic_code in interpro_results_allGeneticCodes[intron] and frame in interpro_results_allGeneticCodes[intron][genetic_code] and i in interpro_results_allGeneticCodes[intron][genetic_code][frame]:
                    hits = interpro_results_allGeneticCodes[intron][genetic_code][frame][i]
                    # Check if any hit contains the substrings "endonuc", "homing", or "nuclease", ignoring case
                    if any(check_substrings(hit) for hit in hits):
                        # Append the list of InterProScan hits to the ORF's dictionary
                        orf['interProScanHits'] = hits
                        # Add the ORF to the new list
                        new_orfs.append(orf)
                    else:
                        # Append the list of InterProScan hits to the ORF's dictionary in discarded_ORFs_dict
                        discarded_ORFs_dict[intron][genetic_code][frame][i]['interProScanHits'] = hits
                else:
                    # Treat the ORF as if there were hits but they did not contain any of the specified substrings
                    orf['interProScanHits'] = []
            # Replace the old list of ORFs with the new list
            frames[frame] = new_orfs
        # Remove the genetic code if all its frames are empty
        if all(not orfs for orfs in frames.values()):
            del homingEndonucleases_dict[intron][genetic_code]
    # remove the intron if all its genetic codes are empty
    if all(not frames for frames in genetic_codes.values()):
        del homingEndonucleases_dict[intron]

# Iterate over each intron in the discarded_ORFs_dict dictionary
for intron, genetic_codes in list(discarded_ORFs_dict.items()):  # Use list to allow modifying the dictionary during iteration
    # For each intron, iterate over each frame
    for genetic_code, frames in list(genetic_codes.items()):
        for frame, orfs in list(frames.items()):  # Use list to allow modifying the dictionary during iteration
            # Remove the frame if all its ORFs have no InterProScan hits
            if all('interProScanHits' not in orf or not orf['interProScanHits'] for orf in orfs):
                del frames[frame]
        # Remove the genetic code if all its frames are empty
        if all(not orfs for orfs in frames.values()):
            del discarded_ORFs_dict[intron][genetic_code]
    # remove the intron if all its genetic codes are empty
    if all(not frames for frames in genetic_codes.values()):
        del discarded_ORFs_dict[intron]

In [ ]:
homingEndonucleases_dict_reshaped = {}

# Iterate over each intron in homingEndonucleases_dict
for intron, genetic_codes in homingEndonucleases_dict.items():
    # Initialize an empty dict for the current intron. each key will be a genetic code
    homingEndonucleases_dict_reshaped[intron] = {}
    for genetic_code, frames in genetic_codes.items():
        # initialize this element of the dict to an empty list for the current genetic code of the current intron
        homingEndonucleases_dict_reshaped[intron][genetic_code] = []
        # For each intron, iterate over each frame of each genetic code
        for frame, orfs in frames.items():
            # For each frame, iterate over each ORF
            for orf in orfs:
                # Create a new dictionary that is a copy of the ORF's dictionary
                orf_copy = copy.deepcopy(orf)
                # Add a new key to the new dictionary with the name 'frame' and the value of the current frame
                orf_copy['frame'] = frame
                # Append the new dictionary to the list of ORFs for the current intron
                homingEndonucleases_dict_reshaped[intron][genetic_code].append(orf_copy)


We also save a dictionary of hits that matched some other protein family different than endonucleases.

In [ ]:
discarded_ORFs_dict_noDisorderHits = {}

for intron, genetic_codes in discarded_ORFs_dict.items():  
    for genetic_code, frames in genetic_codes.items(): 
        for frame, orfs in frames.items(): 
            for i, orf in enumerate(orfs):
                if not 'interProScanHits' in orf.keys():
                    continue
                hits = orf['interProScanHits']
                new_hits = []
                for j, hit in enumerate(hits):
                    if hit["signature"]["name"] is None or not ("disorder" in hit["signature"]["name"].lower() or "coil" in hit["signature"]["name"].lower()):
                        new_hits.append(hit)
                if new_hits:
                    # we make a copy of orf where we replace the interProScanHits with the new list of hits
                    new_orf = orf.copy()
                    new_orf['interProScanHits'] = new_hits
                    if intron not in discarded_ORFs_dict_noDisorderHits:
                        discarded_ORFs_dict_noDisorderHits[intron] = {}
                    if genetic_code not in discarded_ORFs_dict_noDisorderHits[intron]:
                        discarded_ORFs_dict_noDisorderHits[intron][genetic_code] = {}
                    if frame not in discarded_ORFs_dict_noDisorderHits[intron][genetic_code]:
                        discarded_ORFs_dict_noDisorderHits[intron][genetic_code][frame] = {}
                    discarded_ORFs_dict_noDisorderHits[intron][genetic_code][frame][i] = new_orf


We save these dictionaries for future use.

In [ ]:
with open('homingEndonucleases_dict_reshaped_allGeneticCodes.pkl', 'wb') as f:
    pickle.dump(homingEndonucleases_dict_reshaped, f)
with open('discarded_ORFs_dict_noDisorderHits.pkl', 'wb') as f:
    pickle.dump(discarded_ORFs_dict_noDisorderHits, f)

In [ ]:
with open('homingEndonucleases_dict_reshaped_allGeneticCodes.pkl', 'rb') as f:
    homingEndonucleases_dict_reshaped = pickle.load(f)
with open('discarded_ORFs_dict_noDisorderHits.pkl', 'rb') as f:
    discarded_ORFs_dict_noDisorderHits = pickle.load(f)

In [ ]:
# lets count the total number of homing endonucleases identified. In order to do so, we sum the length of each list stored as the value of each key of homingEndonucleases_dict_reshaped

total_homing_endonucleases = sum(len(value) for value in homingEndonucleases_dict_reshaped.values())

print(f"Total number of putative homing endonucleases: {total_homing_endonucleases}")

print(f"These are located in a total of {len(homingEndonucleases_dict_reshaped)} introns")

We now make a dictionary where each key is a number of endonucleases per intron and each value is a list with the introns that have that number of endonucleases

In [ ]:
endonuclease_counter = Counter(len(value) for value in homingEndonucleases_dict_reshaped.values())

print(endonuclease_counter)

introns_by_number_endonucleases = {f"{count} endonuclease" if count == 1 else f"{count} endonucleases": [] for count in endonuclease_counter.keys()}
for key, value in homingEndonucleases_dict_reshaped.items():
    count = len(value)
    introns_by_number_endonucleases[f"{count} endonuclease" if count == 1 else f"{count} endonucleases"].append(key)

We can extract the key for the intron with 4 detected endonucleases

In [ ]:
print(introns_by_number_endonucleases["4 endonucleases"])

Let's now look for introns that had a large difference between the assigned start/end point and the start/end point of the Infernal hit, and that do not contain a homing endonuclease identified through InterProScan.

In [ ]:
classified_introns_bigDifferenceStartOrEndPoints = {}

distance_threshold = 1000

for key, value in classified_introns.items():
    #check if element boundariesSource contains the substring reverse
    if isinstance(value["infernalHitEndPosition"], int):
        if "reverse" in value['boundariesSource']:
            # check that value['infernalHitStartPosition'] is not a tuple
            if abs(value['infernalHitStartPosition'] - value['endPosition']) > distance_threshold or abs(value['infernalHitEndPosition'] - value['startPosition']) > distance_threshold:
                classified_introns_bigDifferenceStartOrEndPoints[key] = value
        else:
            if abs(value['infernalHitStartPosition'] - value['startPosition']) > distance_threshold or abs(value['infernalHitEndPosition'] - value['endPosition']) > distance_threshold:
                classified_introns_bigDifferenceStartOrEndPoints[key] = value
    else:
        anyInfernalBoundariesNoBigDifference = True
        list_of_boundaries = list(zip(value['infernalHitStartPosition'], value['infernalHitEndPosition']))
        # initialize a list of the same length as list_of_boundaries
        infernalBoundariesNoBigDifference = [True] * len(list_of_boundaries)
        for i in range(len(list_of_boundaries)):
            start, end = list_of_boundaries[i]
            if "reverse" in value['boundariesSource']:
                if abs(start - value['endPosition']) > distance_threshold or abs(end - value['startPosition']) > distance_threshold:
                    infernalBoundariesNoBigDifference[i] = False
            else:
                if abs(start - value['startPosition']) > distance_threshold or abs(end - value['endPosition']) > distance_threshold:
                    infernalBoundariesNoBigDifference[i] = False
        if not any(infernalBoundariesNoBigDifference):
            classified_introns_bigDifferenceStartOrEndPoints[key] = value

# get the keys of classified_introns_bigDifferenceStartOrEndPoints that are not present in homingEndonucleases_dict_reshaped

introns_bigDifference_not_in_homingEndonucleases_dict_reshaped = set(classified_introns_bigDifferenceStartOrEndPoints.keys()) - set(homingEndonucleases_dict_reshaped.keys())

Then, let's get the candidate ORFs for these introns (with all possible genetic codes).

In [ ]:

# now lets get the entries of orf_dict_all_genetic_codes corresponding only to the keys in keys_not_in_homingEndonucleases_dict_reshaped

orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE = {key: value for key, value in orf_dict_all_genetic_codes.items() if key in introns_bigDifference_not_in_homingEndonucleases_dict_reshaped}

orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE_flattened = {}

for key, value in orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE.items():
    for genetic_code, frames in value.items():
        for frame, orfs in frames.items():
            for i, orf in enumerate(orfs):
                orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE_flattened[key + f"_geneticCode_{genetic_code}_frame_{frame}_orfNumber_{i}"] = orf

We will try to identify additional homing endonucleases by running DELTA-BLAST on these ORFs

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def run_deltablast(sequence, id):
    # Write the sequence to a temporary file
    with tempfile.NamedTemporaryFile(delete=False) as temp:
        sequence_record = SeqRecord(Seq(sequence), id=id)
        SeqIO.write(sequence_record, temp.name, "fasta")

        # Create a unique temporary file for the BLAST results
        with tempfile.NamedTemporaryFile(delete=False) as temp_blast_results:
            blastn_cline = NcbideltablastCommandline(query=temp.name, db="/path/to/nr/database/nr", outfmt=5, out=temp_blast_results.name, rpsdb="/path/to/cdd_delta_db/cdd_delta", cmd="/path/to/ncbi-blast/bin/deltablast")
            stdout, stderr = blastn_cline()

        # Parse the BLAST results
        result_handle = open(temp_blast_results.name)
        blast_record = NCBIXML.read(result_handle)
        return id, blast_record

# Step 3: Perform BLAST search for each ORF of each frame in each genetic code for all introns
deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE = {}

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(run_deltablast, orf['sequence'], key): key for key, orf in orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE_flattened.items()}
    progress = tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="BLASTs completed")
    for future in progress:
        key = futures[future]
        try:
            _, blast_record = future.result()
            deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE[key] = blast_record
        except Exception as exc:
            print(f'{key} generated an exception: {exc}')

In [ ]:
with open('deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE.pkl', 'wb') as f:
    pickle.dump(deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE, f)

In [ ]:
with open('deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE.pkl', 'rb') as f:
    deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE = pickle.load(f)

We can now make a data frame combining all identified homing endonucleases. First, let's make a dataframe only with HEGs from InterProScan

In [ ]:
# lets initialize a new pandas dataframe with columns intronID,GenBankID,startPositionInIntron,endPositionInIntron,putativeSequence,geneticCode,readingFrame,annotationOrigin
homingEndonucleasesDF_interProScan_raw = pd.DataFrame(columns=['intronID', 'GenBankID', 'startPositionInIntron', 'endPositionInIntron', 'putativeSequence', 'geneticCode', 'readingFrame', 'annotationOrigin'])

def check_substrings(d):
    # Check if the current dictionary contains the substrings
    if any("endonuc" in value.lower() or "homing" in value.lower() or "nuclease" in value.lower() 
           for value in d.values() if isinstance(value, str)):
        return True
    # Recursively check the nested dictionaries
    return any(check_substrings(value) for value in d.values() if isinstance(value, dict)) 

for intron in homingEndonucleases_dict_reshaped:
    intron_subcellular_location = classified_introns[intron]["organelle"]
    if "plastid" in intron_subcellular_location:
        expected_genetic_code = translation_tables_from_NCBI_taxonomy_parsed[intron]["plastidic"]
        other_genetic_codes = [translation_tables_from_NCBI_taxonomy_parsed[intron]["general"], translation_tables_from_NCBI_taxonomy_parsed[intron]["mitochondrial"]]
    elif "mitochondr" in intron_subcellular_location:
        expected_genetic_code = translation_tables_from_NCBI_taxonomy_parsed[intron]["mitochondrial"]
        other_genetic_codes = [translation_tables_from_NCBI_taxonomy_parsed[intron]["general"], translation_tables_from_NCBI_taxonomy_parsed[intron]["plastidic"]]
    else:
        expected_genetic_code = translation_tables_from_NCBI_taxonomy_parsed[intron]["general"]
        other_genetic_codes = [translation_tables_from_NCBI_taxonomy_parsed[intron]["mitochondrial"], translation_tables_from_NCBI_taxonomy_parsed[intron]["plastidic"]]
    interproscan_hegs_current_intron = homingEndonucleases_dict_reshaped[intron]
    for genetic_code in interproscan_hegs_current_intron:
        hegs_current_genetic_code = interproscan_hegs_current_intron[genetic_code]
        for heg in hegs_current_genetic_code:
            if(genetic_code == expected_genetic_code):
                annotationOrigin = "InterProScan hit with expected genetic code for organism and subcellular location"
            elif(genetic_code in other_genetic_codes):
                annotationOrigin = "InterProScan hit with expected genetic code for organism but not subcellular location"
            else:
                annotationOrigin = "InterProScan hit with unexpected genetic code"
            new_row = pd.DataFrame({
                'intronID': [intron],
                'GenBankID': [classified_introns[intron]["GenBankID"]],
                'startPositionInIntron': [heg['startInIntronSeq']],
                'endPositionInIntron': [heg['endInIntronSeq']],
                'putativeSequence': [heg['sequence']],
                'geneticCode': [genetic_code],
                'readingFrame': [heg['frame']],
                'annotationOrigin': [annotationOrigin]
            })
            homingEndonucleasesDF_interProScan_raw = pd.concat([homingEndonucleasesDF_interProScan_raw, new_row], ignore_index=True)

introns_with_hegs_from_interproscan = homingEndonucleasesDF_interProScan_raw['intronID'].unique()

homingEndonucleasesDF_InterProScan = pd.DataFrame(columns=['intronID', 'GenBankID', 'startPositionInIntron', 'endPositionInIntron', 'putativeSequence', 'geneticCode', 'readingFrame', 'annotationOrigin'])

for intron in introns_with_hegs_from_interproscan:
    subDF_hegs = homingEndonucleasesDF_interProScan_raw[homingEndonucleasesDF_interProScan_raw['intronID'] == intron]
    hegs_expected_genetic_code = subDF_hegs[subDF_hegs['annotationOrigin'] == "InterProScan hit with expected genetic code for organism and subcellular location"]
    hegs_other_organelle_genetic_code = subDF_hegs[subDF_hegs['annotationOrigin'] == "InterProScan hit with expected genetic code for organism but not subcellular location"]
    hegs_unexpected_genetic_code = subDF_hegs[subDF_hegs['annotationOrigin'] == "InterProScan hit with unexpected genetic code"]
    # for hegs_other_organelle_genetic_code, we need to cluster the entries that have the same readingFrame, startPositionInIntron and endPositionInIntron 
    hegs_other_organelle_genetic_code_mergedSameSeq = hegs_other_organelle_genetic_code.groupby(['readingFrame', 'startPositionInIntron', 'endPositionInIntron']).agg({
        'intronID': 'first',
        'GenBankID': 'first',
        'putativeSequence': 'first',
        'geneticCode': lambda x: ",".join(map(str, x)),
        'annotationOrigin': 'first'
    }).reset_index()
    # we do the same for hegs_unexpected_genetic_code
    hegs_unexpected_genetic_code_mergedSameSeq = hegs_unexpected_genetic_code.groupby(['readingFrame', 'startPositionInIntron', 'endPositionInIntron']).agg({
        'intronID': 'first',
        'GenBankID': 'first',
        'putativeSequence': 'first',
        'geneticCode': lambda x: ",".join(map(str, x)),
        'annotationOrigin': 'first'
    }).reset_index()
    # then add to the new DF
    homingEndonucleasesDF_InterProScan = pd.concat([homingEndonucleasesDF_InterProScan, hegs_expected_genetic_code, hegs_other_organelle_genetic_code_mergedSameSeq, hegs_unexpected_genetic_code_mergedSameSeq], ignore_index=True)


Next, we make a dataframe with HEGs from DELTA-BLAST. When using DELTA-BLAST hits to identify HEGs, we will only consider hits with an e-value lower than 0.1

In [ ]:
homingEndonucleasesDF_DeltaBlast_raw = pd.DataFrame(columns=['intronID', 'GenBankID', 'startPositionInIntron', 'endPositionInIntron', 'putativeSequence', 'geneticCode', 'readingFrame', 'annotationOrigin'])
e_value_threshold = 0.1

for key in deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE:
    deltablast_hit = deltablast_results_orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE[key]
    intronID = re.match(r"(.*)_geneticCode", key).group(1)
    geneticCode = int(re.search(r"geneticCode_(\d+)_frame", key).group(1))
    frame = int(re.search(r"frame_(\d+)_orfNumber", key).group(1))
    orfNumber = int(re.search(r"orfNumber_(\d+)", key).group(1))
    intron_subcellular_location = classified_introns[intron]["organelle"]
    if "plastid" in intron_subcellular_location:
        expected_genetic_code = translation_tables_from_NCBI_taxonomy_parsed[intron]["plastidic"]
        other_genetic_codes = [translation_tables_from_NCBI_taxonomy_parsed[intron]["general"], translation_tables_from_NCBI_taxonomy_parsed[intron]["mitochondrial"]]
    elif "mitochondr" in intron_subcellular_location:
        expected_genetic_code = translation_tables_from_NCBI_taxonomy_parsed[intron]["mitochondrial"]
        other_genetic_codes = [translation_tables_from_NCBI_taxonomy_parsed[intron]["general"], translation_tables_from_NCBI_taxonomy_parsed[intron]["plastidic"]]
    else:
        expected_genetic_code = translation_tables_from_NCBI_taxonomy_parsed[intron]["general"]
        other_genetic_codes = [translation_tables_from_NCBI_taxonomy_parsed[intron]["mitochondrial"], translation_tables_from_NCBI_taxonomy_parsed[intron]["plastidic"]]
    if(genetic_code == expected_genetic_code):
        annotationOrigin = "DeltaBLAST hit with expected genetic code for organism and subcellular location"
    elif(genetic_code in other_genetic_codes):
        annotationOrigin = "DeltaBLAST hit with expected genetic code for organism but not subcellular location"
    else:
        annotationOrigin = "DeltaBLAST hit with unexpected genetic code"
    length_alignment_largest_hit = 0
    for alignment in deltablast_hit.alignments:
        title = alignment.title.lower()
        length_alignment = alignment.length
        # we want to extract the hit ID, which will be the substring contained between | characters in alignment.hit_id
        hit_id = re.search(r"\|(.*)\|", alignment.hit_id).group(1)
        if ("endonuc" in title or "homing" in title or "nuclease" in title):
            for hsp in alignment.hsps:
                if hsp.expect < e_value_threshold and length_alignment > length_alignment_largest_hit:
                    length_alignment_largest_hit = length_alignment
                    annotationOrigin_with_hitID = annotationOrigin + f" with RefSeq ID {hit_id}"
                    break
    if(length_alignment_largest_hit > 0):
        new_row = pd.DataFrame({
            'intronID': [intronID],
            'GenBankID': [classified_introns[intronID]["GenBankID"]],
            'startPositionInIntron': [orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE_flattened[key]['startInIntronSeq']],
            'endPositionInIntron': [orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE_flattened[key]['endInIntronSeq']],
            'putativeSequence': [orf_dict_all_genetic_codes_intronsBigDiff_and_no_HE_flattened[key]['sequence']],
            'geneticCode': [geneticCode],
            'readingFrame': [frame],
            'annotationOrigin': [annotationOrigin_with_hitID]
        })
        homingEndonucleasesDF_DeltaBlast_raw = pd.concat([homingEndonucleasesDF_DeltaBlast_raw, new_row], ignore_index=True)

introns_with_hegs_from_deltablast = homingEndonucleasesDF_DeltaBlast_raw['intronID'].unique()

homingEndonucleasesDF_DeltaBLAST = pd.DataFrame(columns=['intronID', 'GenBankID', 'startPositionInIntron', 'endPositionInIntron', 'putativeSequence', 'geneticCode', 'readingFrame', 'annotationOrigin'])

for intron in introns_with_hegs_from_deltablast:
    subDF_hegs = homingEndonucleasesDF_DeltaBlast_raw[homingEndonucleasesDF_DeltaBlast_raw['intronID'] == intron]
    hegs_expected_genetic_code = subDF_hegs[subDF_hegs['annotationOrigin'].str.startswith("DeltaBLAST hit with expected genetic code for organism and subcellular location")]
    hegs_other_organelle_genetic_code = subDF_hegs[subDF_hegs['annotationOrigin'].str.startswith("DeltaBLAST hit with expected genetic code for organism but not subcellular location")]
    hegs_unexpected_genetic_code = subDF_hegs[subDF_hegs['annotationOrigin'].str.startswith("DeltaBLAST hit with unexpected genetic code")]
    hegs_other_organelle_genetic_code_mergedSameSeq = hegs_other_organelle_genetic_code.groupby(['readingFrame', 'startPositionInIntron', 'endPositionInIntron']).agg({
        'intronID': 'first',
        'GenBankID': 'first',
        'putativeSequence': 'first',
        'geneticCode': lambda x: ",".join(map(str, x)),
        'annotationOrigin': 'first'
    }).reset_index()
    hegs_unexpected_genetic_code_mergedSameSeq = hegs_unexpected_genetic_code.groupby(['readingFrame', 'startPositionInIntron', 'endPositionInIntron']).agg({
        'intronID': 'first',
        'GenBankID': 'first',
        'putativeSequence': 'first',
        'geneticCode': lambda x: ",".join(map(str, x)),
        'annotationOrigin': 'first'
    }).reset_index()
    homingEndonucleasesDF_DeltaBLAST = pd.concat([homingEndonucleasesDF_DeltaBLAST, hegs_expected_genetic_code, hegs_other_organelle_genetic_code_mergedSameSeq, hegs_unexpected_genetic_code_mergedSameSeq], ignore_index=True)

Finally, we merge them.

In [ ]:
homingEndonucleasesDF = pd.concat([homingEndonucleasesDF_InterProScan, homingEndonucleasesDF_DeltaBLAST], ignore_index=True)
homingEndonucleasesDF.to_csv("homingEndonucleasesDF.csv", index=False)

In [ ]:
homingEndonucleasesDF = pd.read_csv("homingEndonucleasesDF.csv")

Now we are going to compare the histogram of lengths for all introns and for introns where homing endonucleases were detected

In [ ]:
# Extract the sizes of the introns from the classified_introns dictionary
all_intron_sizes = [min(len(intron["sequence"]), 5000) for intron in classified_introns.values()]

# Extract the sizes of the introns with HEGs
introns_with_HEG  = homingEndonucleasesDF['intronID'].unique()
homing_endonuclease_intron_sizes = [min(len(classified_introns[intron]["sequence"]), 5000) for intron in introns_with_HEG if intron in classified_introns]

min_size = 0
max_size = 3000

# Create a histogram for the distribution of intron sizes of all introns
plt.rc('font', family='Arial', size=8)
plt.figure(figsize=(15/2.54, 7.5/2.54))

plt.hist(all_intron_sizes, bins=60, alpha=0.5, color="blue", range=(min_size, max_size), label='All introns', density=True)

# Create a histogram for the distribution of intron sizes of homing endonuclease introns
plt.hist(homing_endonuclease_intron_sizes, bins=60, alpha=0.5, color="red", range=(min_size, max_size), label='Introns with homing endonucleases', density=True)

plt.xlabel('Intron size')
plt.ylabel('Density')
plt.legend()

# Set xticks
xticks = np.arange(min_size, max_size+1, 500)
xticks = np.append(xticks, max_size)
plt.xticks(xticks, [str(int(xtick)) if xtick < max_size else '>3000' for xtick in xticks])

plt.xlim(min_size, max_size)

plt.savefig('Figure1d.png', dpi=300, bbox_inches='tight')
plt.show()

Now we are going to perform secondary structure predictions. First let's set up the arnie.conf file

In [ ]:
!echo "eternafold: /path/to/EternaFold/src" > arnie.conf
!echo "vienna_2: /path/to/ViennaRNA/bin" >> arnie.conf
!echo "nupack: /path/to/nupack3/bin" >> arnie.conf
!echo "contrafold_2: /path/to/contrafold-se/src" >> arnie.conf
!echo "rnastructure: /path/to/RNAstructure/exe" >> arnie.conf
!echo "rnasoft: /path/to/MultiRNAFold" >> arnie.conf
!echo "linearfold: /path/to/LinearFold/bin" >> arnie.conf
!echo "linearpartition: /path/to/LinearPartition/bin" >> arnie.conf
!echo "spotrna: /path/to/SPOT-RNA" >> arnie.conf
!echo "ipknot: /path/to/ipknot/bin" >> arnie.conf
!echo "TMP: /path/to/tmp/folder/" >> arnie.conf

os.environ["ARNIEFILE"] = f'/path/to/arnie.conf'
os.environ["DATAPATH"] = f'/path/to/RNAstructure/data_tables'

import arnie

And now we run predictions of secondary structure and base-pairing probabilities for all introns with all available software. We start with mean free energy (MFE) secondary structure predictions.

In [ ]:
from arnie.mfe import mfe

symbol_mapping = {
    'N': 'A',
    'S': 'C',
    'Y': 'C',
    'B': 'C',
    'R': 'A',
    'D': 'A',
    'W': 'A',
    'K': 'G',
    'V': 'A',
    'H': 'A',
    'M': 'A',
    'T': 'U'
}

def process_intron(key, value):
    sequence = value['sequence']
    for non_standard, standard in symbol_mapping.items():
        sequence = sequence.replace(non_standard, standard)
    with tempfile.TemporaryDirectory() as tmpdir:
        os.chdir(tmpdir)
        try:
            eternafold_MFE_structure = mfe(sequence, package='eternafold')
        except Exception as e:
            print(f"Error processing intron {key} with eternafold: {e}")
            eternafold_MFE_structure = e
        try:
            vienna_MFE_structure = mfe(sequence, package='vienna_2', DEBUG=True)
        except Exception as e:
            print(f"Error processing intron {key} with vienna_2: {e}")
            vienna_MFE_structure = e
        try:
            contrafold_MFE_structure = mfe(sequence, package='contrafold_2', viterbi=True)
        except Exception as e:
            print(f"Error processing intron {key} with contrafold_2: {e}")
            contrafold_MFE_structure = e
        try:
            rnastructure_MFE_structure = mfe(sequence, package='rnastructure', pseudo=False)
        except Exception as e:
            print(f"Error processing intron {key} with rnastructure: {e}")
            rnastructure_MFE_structure = e
    return key, eternafold_MFE_structure, vienna_MFE_structure, contrafold_MFE_structure, rnastructure_MFE_structure

eternafold_MFE_secondary_structures = {}
vienna_MFE_secondary_structures = {}
contrafold_MFE_secondary_structures = {}
rnastructure_MFE_secondary_structures = {}

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = {executor.submit(process_intron, key, value): key for key, value in classified_introns.items()}
    pbar = tqdm(total=len(futures), desc="Processing introns", dynamic_ncols=True)
    for future in concurrent.futures.as_completed(futures):
        key, eternafold_MFE_structure, vienna_MFE_structure, contrafold_MFE_structure, rnastructure_MFE_structure = future.result()
        eternafold_MFE_secondary_structures[key] = eternafold_MFE_structure
        vienna_MFE_secondary_structures[key] = vienna_MFE_structure
        contrafold_MFE_secondary_structures[key] = contrafold_MFE_structure
        rnastructure_MFE_secondary_structures[key] = rnastructure_MFE_structure
        pbar.update(1)
    pbar.close()

with open('results_secondary_structures_MFE.pkl', 'wb') as f:
    pickle.dump({
        'eternafold': eternafold_MFE_secondary_structures,
        'vienna': vienna_MFE_secondary_structures,
        'contrafold': contrafold_MFE_secondary_structures,
        'rnastructure': rnastructure_MFE_secondary_structures
    }, f)

In [ ]:
# we can read back the MFE secondary structures

with open('results_secondary_structures_MFE.pkl', 'rb') as f:
    MFE_secondary_structures = pickle.load(f)

eternafold_MFE_secondary_structures = MFE_secondary_structures['eternafold']
vienna_MFE_secondary_structures = MFE_secondary_structures['vienna']
contrafold_MFE_secondary_structures = MFE_secondary_structures['contrafold']
rnastructure_MFE_secondary_structures = MFE_secondary_structures['rnastructure']

Next we are going to calculate base pair probability (BPP) matrixes for all introns with each package. Note this will take a long time, and require large amounts of RAM.

In [ ]:
from arnie.bpps import bpps

symbol_mapping = {
    'N': 'A',
    'S': 'C',
    'Y': 'C',
    'B': 'C',
    'R': 'A',
    'D': 'A',
    'W': 'A',
    'K': 'G',
    'V': 'A',
    'H': 'A',
    'M': 'A',
    'T': 'U'
}

def process_intron_bpps(key, value):
    sequence = str(value['sequence'])
    for non_standard, standard in symbol_mapping.items():
        sequence = sequence.replace(non_standard, standard)
    with tempfile.TemporaryDirectory() as tmpdir:
        os.chdir(tmpdir)
        try:
            eternafold_BPPS = bpps(sequence, package='eternafold')
        except Exception as e:
            print(f"Error processing intron {key} with eternafold: {e}")
            eternafold_BPPS = e
        try:
            vienna_BPPS = bpps(sequence, package='vienna_2')
        except Exception as e:
            print(f"Error processing intron {key} with vienna_2: {e}")
            vienna_BPPS = e
        try:
            contrafold_BPPS = bpps(sequence, package='contrafold_2')
        except Exception as e:
            print(f"Error processing intron {key} with contrafold_2: {e}")
            contrafold_BPPS = e
        try:
            rnastructure_BPPS = bpps(sequence, package='rnastructure')
        except Exception as e:
            print(f"Error processing intron {key} with rnastructure: {e}")
            rnastructure_BPPS = e
        try:
            rnasoft_BPPS = bpps(sequence, package='rnasoft')
        except Exception as e:
            print(f"Error processing intron {key} with rnasoft: {e}")
            rnasoft_BPPS = e
        try:
            nupack_BPPS = bpps(sequence, package='nupack')
        except Exception as e:
            print(f"Error processing intron {key} with nupack: {e}")
            nupack_BPPS = e
    return key, eternafold_BPPS, vienna_BPPS, contrafold_BPPS, rnastructure_BPPS, rnasoft_BPPS, nupack_BPPS

eternalfold_BPPS_matrixes = {}
vienna_BPPS_matrixes = {}
contrafold_BPPS_matrixes = {}
rnastructure_BPPS_matrixes = {}
rnasoft_BPPS_matrixes = {}
nupack_BPPS_matrixes = {}

eternalfold_BPPS_matrixes_errors = {}
vienna_BPPS_matrixes_errors = {}
contrafold_BPPS_matrixes_errors = {}
rnastructure_BPPS_matrixes_errors = {}
rnasoft_BPPS_matrixes_errors = {}
nupack_BPPS_matrixes_errors = {}

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = {executor.submit(process_intron_bpps, key, value): key for key, value in classified_introns.items()}
    pbar = tqdm(total=len(futures), desc="Processing introns", dynamic_ncols=True)
    for future in concurrent.futures.as_completed(futures):
        key, eternafold_BPPS, vienna_BPPS, contrafold_BPPS, rnastructure_BPPS, rnasoft_BPPS, nupack_BPPS = future.result()
        if isinstance(eternafold_BPPS, Exception):
            eternalfold_BPPS_matrixes_errors[key] = eternafold_BPPS
        else:
            with open(f"BPPS_all/eternafold/{key}_eternafold.npy", "wb") as f:
                np.save(f, eternafold_BPPS)
        if isinstance(vienna_BPPS, Exception):
            vienna_BPPS_matrixes_errors[key] = vienna_BPPS
        else:
            with open(f"BPPS_all/vienna/{key}_vienna.npy", "wb") as f:
                np.save(f, vienna_BPPS)
        if isinstance(contrafold_BPPS, Exception):
            contrafold_BPPS_matrixes_errors[key] = contrafold_BPPS
        else:
            with open(f"BPPS_all/contrafold/{key}_contrafold.npy", "wb") as f:
                np.save(f, contrafold_BPPS)
        if isinstance(rnastructure_BPPS, Exception):
            rnastructure_BPPS_matrixes_errors[key] = rnastructure_BPPS
        else:
            with open(f"BPPS_all/rnastructure/{key}_rnastructure.npy", "wb") as f:
                np.save(f, rnastructure_BPPS)
        if isinstance(rnasoft_BPPS, Exception):
            rnasoft_BPPS_matrixes_errors[key] = rnasoft_BPPS
        else:
            with open(f"BPPS_all/rnasoft/{key}_rnasoft.npy", "wb") as f:
                np.save(f, rnasoft_BPPS)
        if isinstance(nupack_BPPS, Exception):
            nupack_BPPS_matrixes_errors[key] = nupack_BPPS
        else:
            with open(f"BPPS_all/nupack/{key}_nupack.npy", "wb") as f:
                np.save(f, nupack_BPPS)
        pbar.update(1)
    pbar.close()

BPPs run successfully with all software for all introns except for a set of 87 introns, which fail for RNAsoft.

In [ ]:
failed_keys_rnastructure_BPPS_matrixes = list(rnastructure_BPPS_matrixes_errors.keys())

Next, we will classify the introns into subtypes. First, we need to recreate the subgroup-specific CMs from the multiple sequence alignments (in Stockholm format). We first run cmbuild on the MSA and then run cmcalibrate on the resulting CM.

In [ ]:
subtypes_MSAs_folder = "/path/to/input/MSAs_subgroups"
output_CMs_folder = "/path/to/output/CMs_subgroups"
cmbuild_path = "/path/to/cmbuild/bin/cmbuild"
cmcalibrate_path = "/path/to/cmcalibrate/bin/cmcalibrate"
num_cpus = 120

# lets iterate over the different sto files in subtypes_MSAs_folder
sto_files = glob.glob(subtypes_MSAs_folder + "/*.sto")
for MSA_file in sto_files:
    # Define the commands
    output_cm_filename = os.path.basename(MSA_file).replace('.sto', '.cm')
    cmbuild_command = f"{cmbuild_path} {output_CMs_folder}/{output_cm_filename} {MSA_file}"
    cmcalibrate_command = f"{cmcalibrate_path} --cpu {num_cpus} {output_CMs_folder}/{output_cm_filename}"
    # Run the commands
    print(f"Processing cmbuild for {MSA_file}")
    process = subprocess.Popen(cmbuild_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print(f"Processing cmcalibrate for {MSA_file}")
    process = subprocess.Popen(cmcalibrate_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

Next we prepare the database of CMs against which the intron sequences will be run. 

In [ ]:
# first we need to concatenate all the CMs into a single file

output_cm_filename = "/path/to/output/CMs_subgroups/all_subgroups.cm"
cm_files = glob.glob(output_CMs_folder + "/*.cm")
cat_command = f"cat {' '.join(cm_files)} > {output_cm_filename}"
process = subprocess.Popen(cat_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# now we run cmpress on the concatenated CMs

cmpress_path = "/path/to/cmpress/bin/cmpress"
cmpress_command = f"{cmpress_path} {output_cm_filename}"
process = subprocess.Popen(cmpress_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

We will now run cmscan of each intron against the database of CMs, and assign a subtype to each intron based on the CM that gives the best hit.

In [ ]:
intron_sequences = [intron["sequence"] for intron in classified_introns.values()]
intron_sequences_RNA = [Seq(seq.replace("T", "U")) for seq in intron_sequences]

In [ ]:
num_threads = 20

cmscan_path = "/path/to/cmscan/bin/cmscan"

cmscan_runs_path = "/path/to/output/cmscan_runs"

def process_intron(args):
    i, intron_seq, cmscan_path, cmscan_runs_path, output_cm_filename, num_cpus = args
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as fasta_file:
        fasta_file.write(f">intron_{list(classified_introns.keys())[i]}\n{intron_seq}\n")
        fasta_file_path = fasta_file.name
    output_tbl_filename = f"{cmscan_runs_path}/intron_{list(classified_introns.keys())[i]}_max.tbl"
    cmscan_command = f"{cmscan_path} --cpu {num_cpus} --max --tblout {output_tbl_filename} {output_cm_filename} {fasta_file_path}"
    process = subprocess.Popen(cmscan_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    os.remove(fasta_file_path)

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    args = [(i, seq, cmscan_path, cmscan_runs_path, output_cm_filename, 5) for i, seq in enumerate(intron_sequences_RNA)]
    list(tqdm(executor.map(process_intron, args), total=len(intron_sequences_RNA), desc="Processing introns"))

In [ ]:
def read_cmscan_tbl_file(file_path):
    column_names = ["target name", "accession target", "query name", "accession query", "mdl", "mdl from", "mdl to", "seq from", "seq to", "strand", "trunc", "pass", "gc", "bias", "score", "E-value", "inc", "description of target"]
    df = pd.read_csv(file_path, comment='#', sep='\s+', names = column_names, header=None)
    return df
cmscan_runs_path = "/path/to/output/cmscan_runs"
df_best_hit_subtypes = pd.DataFrame(columns=["best hit subtype", "mdl from", "mdl to", "seq from", "seq to", "strand", "E-value", "bitScore", "intron ID"])

list_intron_keys = list(classified_introns.keys())

for i, intron_seq in enumerate(intron_sequences_RNA):
    tbl_file = f"{cmscan_runs_path}/intron_{list(classified_introns.keys())[i]}_max.tbl"
    #tbl_file = f"{cmscan_runs_path}/intron_{i}_max.tbl"
    df = read_cmscan_tbl_file(tbl_file)
    # some dataframes might have no rows. we need to provide None as the value for all columns in this case
    if df.shape[0] == 0:
        df_best_hit_subtypes.loc[i] = [None, None, None, None, None, None, None, list(classified_introns.keys())[i]]
    else:
        best_hit = df.iloc[0]
        df_best_hit_subtypes.loc[i] = [best_hit["target name"], best_hit["mdl from"], best_hit["mdl to"], best_hit["seq from"], best_hit["seq to"], best_hit["strand"], best_hit["E-value"], best_hit["score"], list_intron_keys[i]]

In [ ]:
df_best_hit_subtypes.to_csv("best_hit_subtypes.csv", index=False)

In [ ]:
df_best_hit_subtypes = pd.read_csv("best_hit_subtypes.csv")

We now want to assign secondary structure elements to each intron based on the structural elements found in the CM consensus secondary structure for the corresponding subtype. We first obtain alignments of each intron against the CM of the corresponding subtype.

In [ ]:
# Define the number of threads
num_threads = 10

cmalign_path = "/path/to/cmalign/bin/cmalign"

intron_sequences = [intron["sequence"] for intron in classified_introns.values()]
intron_sequences_RNA = [Seq(seq.replace("T", "U")) for seq in intron_sequences]

intron_sequences_RNA_dict = {list(classified_introns.keys())[i]: seq for i, seq in enumerate(intron_sequences_RNA)}

best_subtype_alignment_runs_path = "/path/to/output/best_subtype_cmalignments"

cms_subgroups_folder_path = output_CMs_folder

def align_intron_to_best_subtype(args):
    i, intron_seq, best_subtype, cmalign_path, best_subtype_alignment_runs_path, cms_subgroups_folder_path, num_cpus = args
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as fasta_file:
        fasta_file.write(f">intron_{list(classified_introns.keys())[i]}\n{intron_seq}\n")
        fasta_file_path = fasta_file.name
    output_stockholm_filename = f"{best_subtype_alignment_runs_path}/intron_{list(classified_introns.keys())[i]}_alignment_to_CM_subtype_{best_subtype}.tbl"

    # Check if the output file already exists and it has a size larger than 0 bytes
    if os.path.exists(output_stockholm_filename) and os.path.getsize(output_stockholm_filename) > 0:
        print(f"Output file {output_stockholm_filename} already exists. Skipping alignment for this intron.")
        os.remove(fasta_file_path)
        return

    best_subtype_cm_filename = f"{cms_subgroups_folder_path}/{best_subtype}.cm"
    cmalign_command = f"{cmalign_path} --cpu {num_cpus} --ileaved --mxsize 200000 --nonbanded -o {output_stockholm_filename} {best_subtype_cm_filename} {fasta_file_path}"
    process = subprocess.Popen(cmalign_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    os.remove(fasta_file_path)

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    args = [(i, seq, df_best_hit_subtypes.iloc[i,0], cmalign_path, best_subtype_alignment_runs_path, cms_subgroups_folder_path, 6) for i, seq in enumerate(intron_sequences_RNA)]
    list(tqdm(executor.map(align_intron_to_best_subtype, args), total=len(intron_sequences_RNA), desc="Processing alignments of introns to best subtype CM"))


We then read the alignment for each intron and compare it with the annotation file of the corresponding subtype, which contains the annotation of structural elements for each position of the consensus sequence (provided as csv files).

In [ ]:
annotated_consensus_subtypes = dict()

path_annotated_subtype_consensus = "/path/to/CMs_subgroups"

for file in os.listdir(path_annotated_subtype_consensus):
    if file.endswith("_annotated_consensus.csv"):
        subtype = file.split("_")[0]
        # the separator is ; in the csv files
        df = pd.read_csv(f"{path_annotated_subtype_consensus}/{file}", sep=';')        
        annotated_consensus_subtypes[subtype] = df

# and now we iterate over the introns alignments and annotate each with structural elements based on the annotations of the consensus sequences

def annotate_structural_elements(alignment_to_CM, annotated_consensus, subtype):
    consensus_with_gaps = alignment_to_CM.column_annotations["reference_annotation"]
    consensus_ss_with_gaps = alignment_to_CM.column_annotations["secondary_structure"]
    aligned_sequence = str(alignment_to_CM[0].seq)
    annotations = annotated_consensus['Group I intron structural element']
    mapping = {}
    ungapped_pos = 0
    for gapped_pos, char in enumerate(consensus_with_gaps):
        if char != '.' and char != '~':
            mapping[gapped_pos] = ungapped_pos
            ungapped_pos += 1
    sequence_annotations = ["-"] * len(aligned_sequence)
    consensus_SS_ungapped = []
    for gapped_pos, char in enumerate(aligned_sequence):
        if char != '-':
            consensus_SS_ungapped.append(consensus_ss_with_gaps[gapped_pos])
            ungapped_pos = mapping.get(gapped_pos)
            if ungapped_pos is not None:
                sequence_annotations[gapped_pos] = annotations[ungapped_pos] 
    # we remove from sequence_annotations the elements at positions for which the character in aligned_sequence is a -
    sequence_annotations = [sequence_annotations[i] for i, char in enumerate(aligned_sequence) if char != '-']
    # we collapse the annotations into a single string
    sequence_annotations = ",".join(sequence_annotations)
    consensus_SS_ungapped = "".join(consensus_SS_ungapped)
    return sequence_annotations,consensus_SS_ungapped

best_subtype_alignment_runs_path = "/path/to/output/best_subtype_cmalignments"

dict_structural_elements = dict()
dict_consensus_SS = dict()

for intron in classified_introns.keys():
    subtype = df_best_hit_subtypes[df_best_hit_subtypes['intron ID'] == intron].iloc[0,0]
    alignment_file = f"{best_subtype_alignment_runs_path}/intron_{intron}_alignment_to_CM_subtype_{subtype}.tbl"
    alignment = AlignIO.read(alignment_file, "stockholm")
    annotated_consensus = annotated_consensus_subtypes[subtype]
    dict_structural_elements[intron],dict_consensus_SS[intron] = annotate_structural_elements(alignment, annotated_consensus, subtype)

In [ ]:
with open("dict_structural_elements.pkl", "wb") as f:
    pickle.dump(dict_structural_elements, f)

with open("dict_consensus_SS.pkl", "wb") as f:
    pickle.dump(dict_consensus_SS, f)

In [ ]:
with open("dict_structural_elements.pkl", "rb") as f:
    dict_structural_elements = pickle.load(f)

with open("dict_consensus_SS.pkl", "rb") as f:
    dict_consensus_SS = pickle.load(f)

Now let's extract the preceding and following exonic context. Extracted intron sequences already contained the last base of the preceding exon and the first base of the following exon; these are included again in the exonic contexts. Up to 150 bases of each are extracted (if there are not enough bases available, extracted sequences are padded with "-").

In [ ]:
classified_introns_preceding_exons = {}
classified_introns_following_exons = {}
number_bases_preceding_exon = 150
number_bases_following_exon = 150

for key, value in tqdm(classified_introns.items(), total=len(classified_introns)):
    start_position_intron = value['startPosition']
    end_position_intron = value['endPosition'] + 1
    genbankID = value['GenBankID']
    genbank_record = infernal_NT_search_genbank_recs[genbankID]
    if genbank_record.seq.defined:
        genbank_sequence_raw = genbank_record.seq
    else:
        genbank_sequence_raw = seqs_of_records_with_undefined_seqs[genbankID]
    genbank_sequence = genbank_sequence_raw.replace('U', 'T')
    source = value['boundariesSource']
    if("reverseStrand" in source):
        reverseStrand = True
    else:
        reverseStrand = False
    if(reverseStrand):
        following_exon_seq = genbank_sequence_raw[max((start_position_intron - number_bases_following_exon + 1), 0):(start_position_intron+1)].reverse_complement()
        preceding_exon_seq = genbank_sequence_raw[(end_position_intron-1):(end_position_intron + number_bases_preceding_exon - 1)].reverse_complement()
    else:
        following_exon_seq = genbank_sequence_raw[(end_position_intron-1):(end_position_intron + number_bases_following_exon - 1)]
        preceding_exon_seq = genbank_sequence_raw[max((start_position_intron - number_bases_preceding_exon + 1), 0):(start_position_intron+1)]
    if(len(following_exon_seq) < number_bases_following_exon):
        following_exon_seq = following_exon_seq + "-"*(number_bases_following_exon - len(following_exon_seq))
    if(len(preceding_exon_seq) < number_bases_preceding_exon):
        preceding_exon_seq = "-"*(number_bases_preceding_exon - len(preceding_exon_seq)) + preceding_exon_seq
    classified_introns_preceding_exons[key] = preceding_exon_seq
    classified_introns_following_exons[key] = following_exon_seq

In [ ]:
with(open("classified_introns_preceding_exons.pkl", "wb")) as f:
    pickle.dump(classified_introns_preceding_exons, f)

with(open("classified_introns_following_exons.pkl", "wb")) as f:
    pickle.dump(classified_introns_following_exons, f)

In [ ]:
with(open("classified_introns_preceding_exons.pkl", "rb")) as f:
    classified_introns_preceding_exons = pickle.load(f)

with(open("classified_introns_following_exons.pkl", "rb")) as f:
    classified_introns_following_exons = pickle.load(f)

Let us also find the position of each intron that aligns with the 10th position of the group I intron CM. For that, we will run cmalign of RF00028 against the subsequence of the GenBank entry of each intron that led to the original hit.

In [ ]:
# Define the number of threads
num_threads = 3

cmalign_path = "/path/to/cmalign/bin/cmalign"

infernal_hits_region_sequences = [intron["infernalHitRegionSequence"] for intron in classified_introns.values()]
infernal_hits_region_sequences_RNA = [Seq(seq.replace("T", "U")) for seq in infernal_hits_region_sequences]

RF00028_alignment_runs_path = "/path/to/output/RF00028_alignment_runs"

RF00028_CM_path = "/path/to/RF00028.cm"

def align_intron_to_RF00028(args):
    i, infernal_hit_region_sequence, cmalign_path, RF00028_alignment_runs_path, num_cpus = args
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as fasta_file:
        fasta_file.write(f">intron_{list(classified_introns.keys())[i]}\n{infernal_hit_region_sequence}\n")
        fasta_file_path = fasta_file.name
    output_stockholm_filename = f"{RF00028_alignment_runs_path}/intron_{list(classified_introns.keys())[i]}_alignment_to_RF00028_CM.tbl"
    # if output_stockholm_filename exists and has a size larger than 0, then just skip the alignment and simply remove the temporary fasta file
    if os.path.exists(output_stockholm_filename) and os.path.getsize(output_stockholm_filename) > 0:
        os.remove(fasta_file_path)
        return
    cmalign_command = f"{cmalign_path} --cpu {num_cpus} --ileaved --nonbanded --mxsize 20024 -o {output_stockholm_filename} {RF00028_CM_path} {fasta_file_path}"
    process = subprocess.Popen(cmalign_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    os.remove(fasta_file_path)

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    args = [(i, seq, cmalign_path, RF00028_alignment_runs_path, 5) for i, seq in enumerate(infernal_hits_region_sequences_RNA)]
    list(tqdm(executor.map(align_intron_to_RF00028, args), total=len(infernal_hits_region_sequences_RNA), desc="Processing alignments of introns to RF00028 CM"))


In [ ]:
def find_target_letter_position(s, target_leter):
    letter_count = 0
    for i, char in enumerate(s):
        if char.isalpha():
            letter_count += 1
        if letter_count == target_leter:
            return i
    return -1

def find_aligned_position(stockholm_file, target_position):
    alignment = AlignIO.read(stockholm_file, "stockholm")
    intron_sequence = str(alignment[0].seq)
    covariance_model_sequence = alignment.column_annotations["reference_annotation"]
    U10_position_in_model_seq = find_target_letter_position(covariance_model_sequence, target_position)
    U10_position_in_intron_seq_char = intron_sequence[U10_position_in_model_seq]
    if not U10_position_in_intron_seq_char.isalpha():
        return None
    else:
        # count how many letter characters there are in the substring of covariance_model_sequence from the beginning to U10_position_in_model_seq
        U10_position_in_intron_seq = 0
        for i, char in enumerate(intron_sequence[:U10_position_in_model_seq]):
            if char.isalpha():
                U10_position_in_intron_seq += 1
        return U10_position_in_intron_seq

dict_position_aligned_to_10th_position_RF00028 = dict()

for intron in classified_introns.keys():
    stockholm_file = f"{RF00028_alignment_runs_path}/intron_{intron}_alignment_to_RF00028_CM.tbl"
    dict_position_aligned_to_10th_position_RF00028[intron] = find_aligned_position(stockholm_file, 10)

In [ ]:
with(open("position_aligned_to_10th_position_RF00028.pkl", "wb")) as f:
    pickle.dump(dict_position_aligned_to_10th_position_RF00028, f)

In [ ]:
with(open("position_aligned_to_10th_position_RF00028.pkl", "rb")) as f:
    dict_position_aligned_to_10th_position_RF00028 = pickle.load(f)

In [ ]:
dict_boundaries_subtype_CM_hit = dict()

best_subtype_alignment_runs_path = "/path/to/output/best_subtype_cmalignments"

for intron in classified_introns.keys():
    subtype = df_best_hit_subtypes[df_best_hit_subtypes['intron ID'] == intron].iloc[0,0]
    tbl_file = f"{cmscan_runs_path}/intron_{intron}_max.tbl"
    df_tbl_file = read_cmscan_tbl_file(tbl_file)
    source = classified_introns[intron]['boundariesSource']
    if "reverseStrand" in source:
        strand = "reverse"
    else:
        strand = "forward"
    if df_tbl_file.shape[0] != 0:
        best_hit_line = df_tbl_file.iloc[0]
        seq_from = best_hit_line["seq from"]
        seq_to = best_hit_line["seq to"]
        intronStartPosition = classified_introns[intron]["startPosition"]
        intronEndPosition = classified_introns[intron]["endPosition"]
        if strand == "forward":
            best_subtype_cm_hit_start = intronStartPosition + seq_from - 1 + 1
            best_subtype_cm_hit_end = intronStartPosition + seq_to - 1 + 1
        else:
            best_subtype_cm_hit_start = intronEndPosition - seq_from + 1 + 1
            best_subtype_cm_hit_end = intronEndPosition - seq_to + 1 + 1
    else:
        best_subtype_cm_hit_start = None
        best_subtype_cm_hit_end = None
    boundaries = (best_subtype_cm_hit_start, best_subtype_cm_hit_end)
    dict_boundaries_subtype_CM_hit[intron] = boundaries  

In [ ]:
data = []
for key, value in classified_introns.items():
    genbankID = value['GenBankID']
    startPosition = int(value['startPosition']) + 1
    endPosition = int(value['endPosition']) + 1
    intronSequence = str(value['sequence'])
    intronLength = value['length']
    intronSubtype = df_best_hit_subtypes[df_best_hit_subtypes['intron ID'] == key].iloc[0,0]
    source = value['boundariesSource']
    if "reverseStrand" in source:
        strand = "reverse"
    else:
        strand = "forward"
    organism = value['organism']
    organismType = value['organismType']
    subcellularLocation = value['organelle']
    if subcellularLocation == '':
        if organismType == "bacteria":
            subcellularLocation = 'cytoplasm'
        elif organismType == "virus":
            subcellularLocation = 'virion'
        else:
            subcellularLocation = 'nucleus'
    taxID = value['taxID']
    if not isinstance(taxID, int):
        taxID = taxID.split(":")[1]
    taxID = str(taxID)
    precedingExonSequence = str(classified_introns_preceding_exons[key])
    followingExonSequence = str(classified_introns_following_exons[key])
    eternafoldSecondaryStructure = eternafold_MFE_secondary_structures[key]
    viennaSecondaryStructure = vienna_MFE_secondary_structures[key]
    contrafoldSecondaryStructure = contrafold_MFE_secondary_structures[key]
    rnastructureSecondaryStructure = rnastructure_MFE_secondary_structures[key]
    consensusSecondaryStructure = dict_consensus_SS[key]
    consensusStructuralElements = dict_structural_elements[key]
    infernalHitStartPositionItem = value['infernalHitStartPosition']
    infernalHitEndPositionItem = value['infernalHitEndPosition']
    # check if infernalHitStartPosition is a tuple
    if isinstance(infernalHitStartPositionItem, tuple):
        if strand == "forward":
            # then we take the smallest value of the tuple
            infernalHitStartPosition = min(infernalHitStartPositionItem)
        else:
            # we take the largest value of the tuple
            infernalHitStartPosition = max(infernalHitStartPositionItem)
    else:
         infernalHitStartPosition = int(value['infernalHitStartPosition'])
    if isinstance(infernalHitEndPositionItem, tuple):
        if strand == "forward":
            infernalHitEndPosition = max(infernalHitEndPositionItem)
        else:
            infernalHitEndPosition = min(infernalHitEndPositionItem)   
    else:
        infernalHitEndPosition = int(value['infernalHitEndPosition'])
    infernalHitEValue = value['infernalHitEValue']
    infernalHitBitScore = value['infernalHitBitScore']
    infernalSubtypeHitEValue = df_best_hit_subtypes[df_best_hit_subtypes['intron ID'] == key]["E-value"].values[0]
    infernalSubtypeHitBitScore = df_best_hit_subtypes[df_best_hit_subtypes['intron ID'] == key]["bitScore"].values[0]
    if strand == "forward":
        # check if infernalHitU10Position is None
        if dict_position_aligned_to_10th_position_RF00028[key] is None:
            infernalHitU10Position = "-"
        else:
            infernalHitU10Position = infernalHitStartPosition + dict_position_aligned_to_10th_position_RF00028[key]
        infernalSubtypeHitStartPosition = dict_boundaries_subtype_CM_hit[key][0]
        infernalSubtypeHitEndPosition = dict_boundaries_subtype_CM_hit[key][1]
    else:
        if dict_position_aligned_to_10th_position_RF00028[key] is None:
            infernalHitU10Position = "-"
        else:
            infernalHitU10Position = infernalHitEndPosition - dict_position_aligned_to_10th_position_RF00028[key]
        infernalSubtypeHitStartPosition = dict_boundaries_subtype_CM_hit[key][0]
        infernalSubtypeHitEndPosition = dict_boundaries_subtype_CM_hit[key][1]
    data.append([key, genbankID, startPosition, endPosition, intronSequence, intronLength, intronSubtype, strand, organism, organismType, subcellularLocation, taxID, precedingExonSequence, followingExonSequence, eternafoldSecondaryStructure, viennaSecondaryStructure, contrafoldSecondaryStructure, rnastructureSecondaryStructure, consensusSecondaryStructure, consensusStructuralElements, infernalHitStartPosition, infernalHitEndPosition, infernalHitEValue, infernalHitBitScore, infernalHitU10Position, infernalSubtypeHitStartPosition, infernalSubtypeHitEndPosition, infernalSubtypeHitEValue, infernalSubtypeHitBitScore])

intronDatabaseDF = pd.DataFrame(data, columns=['intronID', 'GenBankID', 'startPosition', 'endPosition', 'intronSequence', 'intronLength', 'intronSubtype', 'strand', 'organism', 'organismType', 'subcellularLocation', 'organismTaxID', 'precedingExonSequence', 'followingExonSequence', 'eternafoldSecondaryStructure', 'viennaSecondaryStructure', 'contrafoldSecondaryStructure', 'rnastructureSecondaryStructure', 'consensusSecondaryStructure', 'consensusStructuralElements', 'infernalHitStartPosition', 'infernalHitEndPosition', 'infernalHitEValue', 'infernalHitBitScore', 'infernalHitU10Position', 'infernalSubtypeHitStartPosition', 'infernalSubtypeHitEndPosition', 'infernalSubtypeHitEValue', 'infernalSubtypeHitBitScore'])

intronDatabaseDF.to_csv("intronDatabaseDF.tsv", sep='\t', index=False)

Let's check the distribution of intron subtypes by organism types.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

color_map_subtypes = {'IA1': '#FA8072',  # Salmon
                      'IA2': '#FFA07A',  # LightSalmon
                      'IA3': '#E9967A',  # DarkSalmon
                      'IB1': '#ADD8E6',  # LightBlue
                      'IB2': '#87CEEB',  # SkyBlue
                      'IB3': '#4682B4',  # SteelBlue
                      'IB4': '#4169E1',  # RoyalBlue
                      'IC1': '#74B72E',  
                      'IC2': '#32612D',  # LightGreen
                      'IC3': '#03C04A',  # LawnGreen
                      'ID': '#D8BFD8',  # Thistle
                      'IE2': '#FFD700',  # Gold
                      'IE3': '#FFA500'}  # Orange

cm = 1/2.54
fig, axs = plt.subplots(5, 6, figsize=(28*cm, 15*cm))

axs = axs.flatten()

organism_types = sorted(list(set(intronDatabaseDF['organismType'])))

for i, organism_type in enumerate(organism_types):
    current_df = intronDatabaseDF[intronDatabaseDF['organismType'] == organism_type]
    subtype_counts = current_df['intronSubtype'].value_counts()
    colors = [color_map_subtypes[subtype] for subtype in subtype_counts.index]
    axs[i].pie(subtype_counts, colors=colors, wedgeprops=dict(edgecolor='black', linewidth=0.4), radius=1.3)
    axs[i].set_title(f'{organism_type.capitalize()}', fontsize=10)

for i in range(len(organism_types), len(axs)):
    fig.delaxes(axs[i])

legend_elements = [Patch(facecolor=color_map_subtypes[subtype], label=subtype) for subtype in color_map_subtypes]
fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, 0.05), ncol=7, fontsize=10, frameon=False)

plt.tight_layout()
plt.savefig("/path/to/output/Figure_subtypes_distribution.png", format='png', dpi = 300)
plt.show()

Now let's check the distribution of intron subcellular locations by organism types.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Increase global font size

# Define color map with softer shades
color_map = {'chloroplast': '#98FB98',  # PaleGreen
             'mitochondrion': '#FA8072',  # Salmon
             'nucleus': '#ADD8E6',  # LightBlue
             'other plastids': '#D8BFD8',  # Thistle
             'other': '#FFD700'}  # Gold

cm = 1/2.54
fig, axs = plt.subplots(5, 6, figsize=(28*cm, 15*cm))
# Step 1
def map_locations(location):
    if location == 'plastid' or location =="plastid:chloroplast":
        return 'chloroplast'
    elif location == "mitochondria":
        return 'mitochondrion'
    elif location == "nucleus":
        return 'nucleus'
    elif 'plastid' in location and 'chloroplast' not in location: 
        return 'other plastids'
    else:
        return 'other'

# Flatten the axs array for easy iteration
axs = axs.flatten()

organism_types = sorted(list(set(intronDatabaseDF['organismType'])))

for i, organism_type in enumerate(organism_types):
    current_df = intronDatabaseDF[intronDatabaseDF['organismType'] == organism_type]
    current_df['mappedLocation'] = current_df['subcellularLocation'].apply(map_locations)
    location_counts = current_df['mappedLocation'].value_counts()
    colors = [color_map[loc] for loc in location_counts.index]
    axs[i].pie(location_counts, colors=colors, wedgeprops=dict(edgecolor='black', linewidth=0.4), radius=1.3)
    axs[i].set_title(f'{organism_type.capitalize()}', fontsize=10)

# Remove empty subplots
for i in range(len(organism_types), len(axs)):
    fig.delaxes(axs[i])

legend_elements = [Patch(facecolor=color_map[loc], label=loc) for loc in color_map]
fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, 0.05), ncol=7, fontsize=10, frameon=False)

plt.tight_layout()
plt.savefig("/path/to/output/Figure_organelle_distribution.png", format='png', dpi = 300)
plt.show()

Finally, let's analyze the presence of different structural elements in introns of each subtype

In [ ]:
all_consensus_structural_elements_strings=intronDatabaseDF["consensusStructuralElements"]
unique_consensus_structural_elements=set()
for consensus_structural_elements in all_consensus_structural_elements_strings:
    unique_consensus_structural_elements.update(consensus_structural_elements.split(","))
unique_consensus_structural_elements.remove("-")

In [ ]:
def process_introns_for_structural_elements(intronDatabase, unique_consensus_structural_elements_without_apostrophe):
    intron_dict = {}
    subtypes = intronDatabase['intronSubtype'].unique()

    for subtype in subtypes:
        subtype_df = intronDatabase[intronDatabase['intronSubtype'] == subtype]

        for _, intron in subtype_df.iterrows():
            consensusStructuralElementsString = intron['consensusStructuralElements']
            splitConsensusStructuralElementsString = consensusStructuralElementsString.split(',')
            structuralElementsPresentInThisIntron = []

            for element in unique_consensus_structural_elements_without_apostrophe:
                if element in splitConsensusStructuralElementsString and element + '’' in splitConsensusStructuralElementsString:
                    structuralElementsPresentInThisIntron.append(element)

            intron_dict[intron['intronID']] = structuralElementsPresentInThisIntron

    return intron_dict

In [ ]:
# Define the order of the structural elements
order = ['P1', 'P2', 'P2.1', 'P2.1a', 'P2.1b', 'P3', 'P4', 'P5', 'P5a', 'P5b', 'P5c', 'P5d', 'P6', 'P6a', 'P7', 'P7.1', 'P7.2', 'P8', 'P9', 'P9.0', 'P9.1', 'P9.2', 'P9.3']

# Define the order of the subtypes
subtypes_order = ['IA1', 'IA2', 'IA3', 'IB1', 'IB2', 'IB3', 'IB4', 'IC1', 'IC2', 'IC3', 'ID', 'IE2', 'IE3']

# Initialize figure with a grid of subplots with 4 columns and 4 rows
# Increase the size of the figure by 25%
cm = 1/2.54
fig, axs = plt.subplots(4, 4, figsize=(30*cm, 28*cm))

# Create a colormap

colors = ['#FA8072', #P1
          '#981F2F', #P2s
          '#BA263A',
          '#D53449',
          '#D94558',
          '#FFD700', #P3
          '#851693', #P4
          '#4169E1', #P5s
          '#4682B4',
          '#87CEEB',
          '#ADD8E6',
          '#D0F0F8',
          '#FFA500', #P6s
          '#FFD700',
          '#03C04A', #P7s
          '#32612D',
          '#74B72E',
          '#9381FF', #P8
          '#6E6349', #P9s
          '#87795A',
          '#9D8E6C',
          '#ADA185',
          '#BFB59E']

for i, subtype in enumerate(subtypes_order):
    subtype_df = intronDatabaseDF[intronDatabaseDF['intronSubtype'] == subtype]
    intron_dict = process_introns_for_structural_elements(subtype_df, unique_consensus_structural_elements)

    # Count occurrences of each element
    element_counts = {element: 0 for element in unique_consensus_structural_elements}
    for elements in intron_dict.values():
        for element in elements:
            if element in element_counts:
                element_counts[element] += 1

    # Convert counts to percentages
    total_introns = len(intron_dict)
    element_percentages = {element: (count / total_introns) * 100 for element, count in element_counts.items()}

    # Reorder the keys and values according to the specified order
    ordered_keys = [element for element in order if element in element_percentages]
    ordered_values = [element_percentages[element] for element in ordered_keys]

    # Create bar plot in the corresponding subplot
    ax = axs[i // 4, i % 4]
    # Generate colors from the colormap
    #colors = cmap(np.linspace(0, 1, len(ordered_keys)))
    ax.bar(ordered_keys, ordered_values, color=colors)
    ax.set_ylabel('Percentage of Introns (%)')
    ax.set_title(subtype, fontsize=10)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)  # Rotate x-axis labels for readability

# Remove empty subplots
for i in range(len(subtypes_order), 16):
    fig.delaxes(axs.flatten()[i])

plt.tight_layout()
plt.savefig("/path/to/output/Figure_structural_elements.png", format='png', dpi = 300)
plt.show()